# SETUP

## SETUP

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [34]:
!pip install -U "ray[tune]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 MB 13.4 MB/s eta 0:00:00


In [2]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
!pip install pypots

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 119.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 47.6 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.7 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-6lplh9lz
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-6lplh9lz
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 0d013cf488a722d5a5b3bf657302fa7ca8b6d120
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.7.0-py3-none-any.whl size=12

In [3]:
!pip install pytorch-forecasting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 50.1 MB/s eta 0:00:00


In [4]:
!pip install pytorch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 7.5 MB/s eta 0:00:00


In [5]:
!mkdir -p datasets/knmi_station_data
!cp -r /content/drive/MyDrive/MAGISTERKA/datasets/knmi_station_data ./datasets/

## Imports

In [85]:
from collections import Counter
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from pypots.utils.random import set_random_seed
from pypots.optim import Adam
from pypots.classification import Raindrop, BRITS, GRUD
from pypots.nn.functional import calc_binary_classification_metrics, calc_mse, calc_rmse, calc_mae
from pypots.nn.modules.loss import Criterion, MSE, MAE
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import v2
from torch.utils.data import WeightedRandomSampler
import matplotlib.pyplot as plt
import seaborn as sns

from pytorch_forecasting.data.timeseries import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_forecasting.models import Baseline, TemporalFusionTransformer
from pytorch_forecasting.metrics import MAE, RMSE, MASE

import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.tuner import Tuner
from torchmetrics import Metric

from ray.tune.schedulers import ASHAScheduler
import os
import tempfile


import ray
from ray import tune

In [7]:
BASE_PATH = Path("/content")
DRIVE_PATH = Path("/content/drive/MyDrive/MAGISTERKA")
SAVE_DIR = DRIVE_PATH / "lightning_logs/runs/forecast"

# Prepare df

In [8]:
stations = '249','323', '377'
# stations = '323',
test_station = '215'

In [9]:
def _convert_vv_to_meters(vv_code):
    if pd.isna(vv_code):
        return np.nan

    vv_code = int(vv_code)

    if 0 <= vv_code <= 49:
        return vv_code * 100 + 50
    elif vv_code == 50:
        return 5500
    elif 51 <= vv_code <= 55:
        return np.nan
    elif 56 <= vv_code <= 79:
        return int((vv_code - 56 + 6.5) * 1000)
    elif vv_code == 80:
        return 32500
    elif 81 <= vv_code <= 88:
        return int(32500 + (vv_code - 81) * 5000)
    elif vv_code == 89:
        return 70000
    else:
        return np.nan

def _convert_vvm_to_simple(vv_m):
    if pd.isna(vv_m):
        return np.nan

    if vv_m < 500:
      return 0
    elif vv_m < 1000:
      return 1
    elif vv_m < 2000:
      return 2
    elif vv_m < 5000:
      return 3
    elif vv_m < 10000:
      return 4
    else:
      return 5

def _get_valid_vv_codes() -> list[int]:
    valid_codes = list(range(0, 51))
    valid_codes += list(range(56, 90))
    return valid_codes

def get_vv_one_hot_encoder() -> OneHotEncoder:
    valid_codes = _get_valid_vv_codes()
    categories = [np.array(valid_codes, dtype=np.int32)]
    encoder = OneHotEncoder(categories=categories, handle_unknown='ignore', dtype=np.float32, sparse_output=False)
    encoder.fit(categories[0].reshape(-1, 1))
    return encoder

def prepare_df(path: str) -> pd.DataFrame:
    try:
        header_line_index = -1
        column_names = []
        data_lines_start_index = -1

        # Find the header and its index more efficiently
        with open(path, 'r') as f:
            for i, line in enumerate(f):
                if line.strip().startswith('# STN,YYYYMMDD,'):
                    header_line_index = i
                    column_names = [col.strip() for col in line.strip().lstrip('#').split(',')]
                    data_lines_start_index = header_line_index + 1
                    break

        if header_line_index == -1:
            raise ValueError("Header line not found.")

        # Use pandas.read_csv directly with skiprows and comment character
        # This avoids reading the whole file into a list first for data lines
        # and then joining them back.
        df = pd.read_csv(
            path,
            names=column_names,
            skiprows=data_lines_start_index,
            comment='#',  # Lines starting with '#' will be ignored as comments
            skipinitialspace=True,
            na_values=['       ', '     '] # Add other common missing value representations if needed
        )

        if df.empty:
            raise ValueError("No data found after the header or all data was commented out.")

        # Convert 'HH' to string and zfill, then create 'Timestamp'
        # It's crucial to handle potential NaN values in 'YYYYMMDD' or 'HH'
        # if they are not guaranteed to be present or valid in all rows.
        df['HH'] = df['HH'].astype(int) - 1
        df['HH'] = df['HH'].astype(str).str.zfill(2)
        df['Timestamp'] = pd.to_datetime(df['YYYYMMDD'].astype(str) + df['HH'].astype(str), format="%Y%m%d%H", errors='coerce')

        df.set_index('Timestamp', inplace=True)

        # Columns to drop
        cols_to_drop = ['YYYYMMDD', 'HH']
        df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

        # Convert remaining columns to numeric, efficiently
        # Identify numeric columns once and convert
        # Exclude already processed or known non-numeric columns if necessary
        for col in df.columns:
            # This check is slightly redundant if YYYYMMDD and HH are already dropped,
            # but good for safety if they weren't or if other non-numeric columns exist.
            if df[col].dtype == 'object': # Only attempt conversion if the column is of object type
                try:
                    df[col] = pd.to_numeric(df[col], downcast='signed')
                except ValueError:
                    # Handle or log cases where a column expected to be numeric isn't
                    # For now, we'll coerce, which turns unparseable into NaT/NaN
                    df[col] = pd.to_numeric(df[col], errors='coerce', downcast='signed')
        df['VV_m'] = df['VV'].apply(_convert_vv_to_meters)
        df['VV_s'] = df['VV_m'].apply(_convert_vvm_to_simple)

        return df

    except FileNotFoundError:
        print(f"Error: The file '{path}' was not found.")
        raise
    except ValueError as ve:
        print(f"ValueError: {ve}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

In [10]:
dfs = []
for station in stations:
    df = prepare_df(f"./datasets/knmi_station_data/{station}.txt")
    df = df.set_index('STN', append=True)
    # Check whether VV column has any noy nulls
    nulls = df['VV'].isna().sum()
    dfs.append(df)

train_df = pd.concat(dfs)
train_df = df.reset_index()
train_df.head()

,Timestamp,STN,DD,FH,FF,FX,T,T10N,TD,SQ,...,U,WW,IX,M,R,S,O,Y,VV_m,VV_s
0,2000-01-01 00:00:00,377,220,30,30,50,46,NaN,39,0,...,95,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-01 01:00:00,377,210,30,30,50,47,NaN,41,0,...,96,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-01 02:00:00,377,210,30,30,50,48,NaN,44,0,...,97,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-01 03:00:00,377,200,30,30,50,49,NaN,45,0,...,97,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-01 04:00:00,377,200,30,30,50,50,NaN,46,0,...,97,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88104 entries, 0 to 88103
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Timestamp  88104 non-null  datetime64[ns]
 1   STN        88104 non-null  int64         
 2   DD         88104 non-null  int64         
 3   FH         88104 non-null  int64         
 4   FF         88104 non-null  int64         
 5   FX         88104 non-null  int64         
 6   T          88104 non-null  int64         
 7   T10N       14682 non-null  float64       
 8   TD         88104 non-null  int64         
 9   SQ         88104 non-null  int64         
 10  Q          88104 non-null  int64         
 11  DR         88104 non-null  int64         
 12  RH         88104 non-null  int64         
 13  P          0 non-null      float64       
 14  VV         77850 non-null  float64       
 15  N          77775 non-null  float64       
 16  U          88104 non-null  int64        

In [12]:
dfs = []
for station in [test_station]:
    df = prepare_df(f"./datasets/knmi_station_data/{station}.txt")
    df = df.set_index('STN', append=True)
    # Check whether VV column has any noy nulls
    nulls = df['VV'].isna().sum()
    dfs.append(df)

test_df = pd.concat(dfs)
test_df = df.reset_index()
test_df.head()

,Timestamp,STN,DD,FH,FF,FX,T,T10N,TD,SQ,...,U,WW,IX,M,R,S,O,Y,VV_m,VV_s
0,2015-01-01 00:00:00,215,210.0,50.0,50.0,70.0,27,NaN,8,0,...,87,10.0,7,0.0,0.0,0.0,0.0,0.0,4250.0,3.0
1,2015-01-01 01:00:00,215,220.0,50.0,50.0,70.0,26,NaN,4,0,...,85,10.0,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0
2,2015-01-01 02:00:00,215,200.0,50.0,40.0,80.0,23,NaN,2,0,...,86,NaN,5,0.0,0.0,0.0,0.0,0.0,10500.0,5.0
3,2015-01-01 03:00:00,215,210.0,40.0,40.0,70.0,21,NaN,1,0,...,87,NaN,5,0.0,0.0,0.0,0.0,0.0,10500.0,5.0
4,2015-01-01 04:00:00,215,190.0,50.0,50.0,80.0,19,NaN,2,0,...,88,NaN,5,0.0,0.0,0.0,0.0,0.0,10500.0,5.0


In [13]:
SEQUENCE_LENGTH = 12
STEP_SIZE = 1
TARGET_COLUMN = 'M'
NUMERICAL_COLS = [
    "FH", "FF", "FX", "T", "T10N", "TD", "SQ", "Q", "DR", "RH", "P", "U",
    # "DD"
]
CATEGORICAL_COLS = {
    # "WW", "IX", "VV"
}

vv_encoder = get_vv_one_hot_encoder()


# Pytorch forecasting dataset

In [14]:
def prepare_forecasting_df(df: pd.DataFrame) -> pd.DataFrame:
  df = df.copy()
  df = df[df[TARGET_COLUMN].notna()]

  prepared_dfs = []
  for station in df['STN'].unique():
    station_df = df[df['STN'] == station]
    station_df = station_df.sort_values('Timestamp').reset_index(drop=True)
    valid_times = station_df['Timestamp'].sort_values().reset_index(drop=True)
    time_diffs = valid_times.diff().fillna(pd.Timedelta(seconds=0))
    group = (time_diffs > pd.Timedelta(hours=1)).cumsum()
    station_df['TimeGroup'] = group
    station_df['TimeIdx'] = station_df.groupby('TimeGroup').cumcount()
    prepared_dfs.append(station_df)
  return pd.concat(prepared_dfs, axis=0).fillna(-1)

In [15]:
train_df_v2 = prepare_forecasting_df(train_df)
test_df_v2 = prepare_forecasting_df(test_df)

In [16]:
train_df_v2.head()

,Timestamp,STN,DD,FH,FF,FX,T,T10N,TD,SQ,...,IX,M,R,S,O,Y,VV_m,VV_s,TimeGroup,TimeIdx
0,2003-01-01 00:00:00,377,70,20,20,30,6,-1.0,-12,0,...,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0,0,0
1,2003-01-01 01:00:00,377,0,10,0,20,9,-1.0,-8,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,1
2,2003-01-01 02:00:00,377,0,0,0,10,10,-1.0,-5,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,2
3,2003-01-01 03:00:00,377,220,10,30,40,15,-1.0,-2,0,...,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0,0,3
4,2003-01-01 04:00:00,377,220,30,30,50,15,-1.0,2,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,4


In [17]:
TRAIN_CUTOFF = pd.to_datetime('2021-01-01')
train_dataset = TimeSeriesDataSet(
    data=train_df_v2[train_df_v2['Timestamp'] < TRAIN_CUTOFF],
    time_idx='TimeIdx',
    target=TARGET_COLUMN,
    group_ids=['STN', 'TimeGroup'],
    min_encoder_length=8,
    max_encoder_length=8,
    min_prediction_length=1,
    max_prediction_length=1,
    time_varying_known_reals=NUMERICAL_COLS,
    add_relative_time_idx=False,
    categorical_encoders={
        'TimeGroup': NaNLabelEncoder(add_nan=True, warn=False),
        'STN': NaNLabelEncoder(add_nan=True, warn=False),
    }
)
validation_dataset = TimeSeriesDataSet.from_dataset(
    train_dataset,
    train_df_v2[train_df_v2['Timestamp'] >= TRAIN_CUTOFF],
    stop_randomization=True,
)
train_dataset

TimeSeriesDataSet[length=60783](
	time_idx='TimeIdx',
	target='M',
	group_ids=['STN', 'TimeGroup'],
	weight=None,
	max_encoder_length=8,
	min_encoder_length=8,
	min_prediction_idx=0,
	min_prediction_length=1,
	max_prediction_length=1,
	static_categoricals=None,
	static_reals=None,
	time_varying_known_categoricals=None,
	time_varying_known_reals=['FH', 'FF', 'FX', 'T', 'T10N', 'TD', 'SQ', 'Q', 'DR', 'RH', 'P', 'U'],
	time_varying_unknown_categoricals=None,
	time_varying_unknown_reals=None,
	variable_groups=None,
	constant_fill_strategy=None,
	allow_missing_timesteps=False,
	lags=None,
	add_relative_time_idx=False,
	add_target_scales=False,
	add_encoder_length=False,
	target_normalizer=GroupNormalizer(
	method='standard',
	groups=None,
	center=True,
	scale_by_group=False,
	transformation=None,
	method_kwargs={}
),
	categorical_encoders={'TimeGroup': NaNLabelEncoder(add_nan=True, warn=False), 'STN': NaNLabelEncoder(add_nan=True, warn=False), '__group_id__STN': NaNLabelEncoder(add_nan=Fals

In [18]:
target = train_df_v2.loc[train_dataset.index.index, TARGET_COLUMN].to_numpy()
# probabilities = np.ones_like(target)
# probabilities[target < 5000] = 3
# probabilities[target < 3000] = 5
# probabilities[target < 1000] = 10

train_dl = train_dataset.to_dataloader(
  batch_size=64,
  # sampler=WeightedRandomSampler(weights=probabilities, num_samples=len(probabilities), replacement=True),
  shuffle=False,
  num_workers=0
)

In [19]:
# target = train_df_v2.loc[validation_dataset.index.index, "VV_m"].to_numpy()
# probabilities = np.ones_like(target)
# probabilities[target < 5000] = 3
# probabilities[target < 3000] = 5
# probabilities[target < 1000] = 10

validation_dl = validation_dataset.to_dataloader(
  batch_size=64 * 10,
  # sampler=WeightedRandomSampler(weights=probabilities, num_samples=len(probabilities), replacement=True),
  shuffle=False,
  num_workers=0,
  drop_last=True
)

# Pypots dataset

In [20]:
train_df_v2 = prepare_forecasting_df(train_df)
test_df_v2 = prepare_forecasting_df(test_df)

train_df_v2.head()

,Timestamp,STN,DD,FH,FF,FX,T,T10N,TD,SQ,...,IX,M,R,S,O,Y,VV_m,VV_s,TimeGroup,TimeIdx
0,2003-01-01 00:00:00,377,70,20,20,30,6,-1.0,-12,0,...,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0,0,0
1,2003-01-01 01:00:00,377,0,10,0,20,9,-1.0,-8,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,1
2,2003-01-01 02:00:00,377,0,0,0,10,10,-1.0,-5,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,2
3,2003-01-01 03:00:00,377,220,10,30,40,15,-1.0,-2,0,...,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0,0,3
4,2003-01-01 04:00:00,377,220,30,30,50,15,-1.0,2,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,4


In [21]:
def df_to_pypots(df: pd.DataFrame, prev_values: int=8, cols: list[str]=NUMERICAL_COLS, target_col: str = 'VV_s'):
  df = df.copy()
  X = []
  y = []
  for _, group in df.groupby(['TimeGroup', 'STN']):
    if len(group) < prev_values:
      continue
    # Sliding window over group and add to X, y
    for i in range(len(group) - prev_values):
      X.append(group.iloc[i:i+prev_values][cols].values)
      y.append(group.iloc[i+prev_values][target_col])
  X = np.array(X)
  y = np.array(y)
  print(X.shape, y.shape)
  return X, y


X_train, y_train = df_to_pypots(
    train_df_v2[train_df_v2['Timestamp'] < TRAIN_CUTOFF],
    prev_values=8,
    target_col=TARGET_COLUMN,
    cols=NUMERICAL_COLS + ['VV_m', 'VV'],
)

X_val, y_val = df_to_pypots(
    train_df_v2[train_df_v2['Timestamp'] >= TRAIN_CUTOFF],
    prev_values=8,
    target_col=TARGET_COLUMN,
    cols=NUMERICAL_COLS + ['VV_m', 'VV'],
)

X_test, y_test = df_to_pypots(
    test_df_v2,
    prev_values=8,
    target_col=TARGET_COLUMN,
    cols=NUMERICAL_COLS + ['VV_m', 'VV'],
)

(60783, 8, 14) (60783,)
(16856, 8, 14) (16856,)
(36925, 8, 14) (36925,)


In [22]:
def balance_dataset(X, y, up_to: int=2000):
  Xs = []
  ys = []
  for v, c in zip(*np.unique(y, return_counts=True)):
    if c < up_to:
      Xs.append(X[y == v])
      ys.append(y[y == v])
    else:
      # shuffle
      indexes = np.random.choice(c, up_to, replace=False)
      Xs.append(X[y == v][indexes])
      ys.append(y[y == v][indexes])
  return np.concatenate(Xs), np.concatenate(ys)

In [23]:
np.unique(y_val, return_counts=True)

(array([0., 1.]), array([16301,   555]))

In [24]:
X_train_bal, y_train_bal = balance_dataset(X_train, y_train, up_to=2000)
X_val_bal, y_val_bal = balance_dataset(X_val, y_val, up_to=600)
X_test_bal, y_test_bal = balance_dataset(X_test, y_test)

In [25]:
np.unique(y_train_bal, return_counts=True), len(y_train_bal)

((array([0., 1.]), array([2000, 1877])), 3877)

In [26]:
np.unique(y_val_bal, return_counts=True)

(array([0., 1.]), array([600, 555]))

In [27]:
# Shuffle datasets
indexes = np.random.permutation(len(X_train_bal))
X_train_bal = X_train_bal[indexes]
y_train_bal = y_train_bal[indexes]

indexes = np.random.permutation(len(X_val_bal))
X_val_bal = X_val_bal[indexes]
y_val_bal = y_val_bal[indexes]

indexes = np.random.permutation(len(X_test_bal))
X_test_bal = X_test_bal[indexes]
y_test_bal = y_test_bal[indexes]



# Deep models

In [88]:
from time import time

def _get_formatted_metric(values: list[float]):
  avg = sum(values) / len(values)
  std = (sum([(v - avg) ** 2 for v in values]) / len(values)) ** 0.5
  return f"{avg:.4f} ± {std:.4f}"


def run_model(get_model, name: str, X_train, y_train, X_val, y_val, X_test, y_test, repeat: int = 5):
  final_res = {
      'accuracy': [],
      'f1': [],
      'precision': [],
      'recall': []
  }
  training_times = []
  for _ in range(repeat):
    model = get_model()
    _start = time()
    model.fit({'X': X_train, 'y': y_train}, {'X': X_val, 'y': y_val})
    _end = time()
    training_times.append(_end - _start)
    results = model.predict({'X': X_test, 'y': y_test})
    prediction = results['classification']
    metrics = calc_binary_classification_metrics(prediction, y_test)
    final_res['accuracy'].append(metrics['accuracy'])
    final_res['f1'].append(metrics['f1'])
    final_res['precision'].append(metrics['precision'])
    final_res['recall'].append(metrics['recall'])

    print(
      f"{name} & {_get_formatted_metric(final_res['accuracy'])} & {_get_formatted_metric(final_res['f1'])} & {_get_formatted_metric(final_res['precision'])} & {_get_formatted_metric(final_res['recall'])} \\"
    )
    print(training_times)
    print(_get_formatted_metric(training_times))


In [79]:
def train_pypots_model(config, model_base_params, model_cls, X_train, y_train, X_val, y_val):
    if "lr" in config:
      optimizer = Adam(lr=config["lr"])
      del config["lr"]
    else:
      optimizer = Adam()

    if "epochs" in config:
      epochs = config["epochs"]
      del config["epochs"]
    else:
      epochs = 10
    model = model_cls(
        **config,
        **model_base_params,
        optimizer=optimizer,
        epochs=1
    )
    if cp := tune.get_checkpoint():
      with cp.as_directory() as cp_directory:
        model.load(
            os.path.join(cp_directory, "checkpoint.pypots")
        )
    for _ in range(epochs):
      model.fit(train_set={'X': X_train, 'y': y_train}, val_set={'X': X_val, 'y': y_val})
      results = model.predict({'X': X_val, 'y': y_val})
      prediction = results['classification']
      metrics = calc_binary_classification_metrics(prediction, y_val)

      with tempfile.TemporaryDirectory() as tmp_dir:
        path = os.path.join(tmp_dir, "checkpoint.pypots")
        model.save(path)
        checkpoint = tune.Checkpoint.from_directory(tmp_dir)
        tune.report(
            metrics={
              "accuracy": metrics['accuracy'],
              "f1": metrics['f1'],
              "precision": metrics['precision'],
              "recall": metrics['recall'],
            },
            checkpoint=checkpoint
        )

In [83]:
scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=10,
    grace_period=2,
    reduction_factor=2
)

tuner = tune.Tuner(
    tune.with_resources(
      tune.with_parameters(
          train_pypots_model,
          model_cls=Raindrop,
          model_base_params={
            "n_steps": X_train_bal.shape[1],
            "n_features": X_train_bal.shape[2],
            "n_classes": len(np.unique(y_train_bal)),
            "batch_size": 64,
            # "patience": 6,
            "num_workers": 0,
            "device": None,
            "model_saving_strategy": None,
            "verbose": True
          },
          X_train=X_train_bal,
          y_train=y_train_bal,
          X_val=X_val_bal,
          y_val=y_val_bal,
      ),
      resources={"cpu": 1}
    ),
    param_space={
        "n_layers": tune.choice([1, 2]),
        "d_ffn": tune.choice([32, 64, 128, 256]),
        "n_heads": tune.choice([1, 2]),
        "d_model": tune.choice([X_train_bal.shape[2] * i for i in range(1, 4+1)]),
        "dropout": tune.choice([0.1, 0.2, 0.3]),
        "lr": tune.loguniform(1e-4, 1e-1),
    },
    tune_config=tune.TuneConfig(
        num_samples=5,
        metric="accuracy",
        mode="max",
        scheduler=scheduler,
    )
)

results = tuner.fit()
best_results = results.get_best_result(metric='accuracy', mode='max')
best_metrics = best_results.metrics
print(f"Best params: {best_results.config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

+---------------------------------------------------------------------------+
| Configuration for experiment     train_pypots_model_2025-06-07_15-13-35   |
+---------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                    |
| Scheduler                        AsyncHyperBandScheduler                  |
| Number of trials                 5                                        |
+---------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_pypots_model_2025-06-07_15-13-35
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_11-22-25_127757_2711/artifacts/2025-06-07_15-13-35/train_pypots_model_2025-06-07_15-13-35/driver_artifacts`

Trial status: 5 PENDING
Current time: 2025-06-07 15:13:35. Total running time: 0s
Logical resource usage: 0/8 CPUs, 0/0 GPUs
+----------------------------------

(pid=121920) 2025-06-07 15:13:41.059873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=121920) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=121920) E0000 00:00:1749309221.125110  121920 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=121920) E0000 00:00:1749309221.145120  121920 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



Trial train_pypots_model_fb786_00004 started with configuration:
+---------------------------------------------------------+
| Trial train_pypots_model_fb786_00004 config             |
+---------------------------------------------------------+
| d_ffn                                               128 |
| d_model                                              56 |
| dropout                                             0.1 |
| lr                                              0.00109 |
| n_heads                                               2 |
| n_layers                                              2 |
+---------------------------------------------------------+
(pid=121926) 
(pid=121926) ████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
(pid=121926) ╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
(pid=121926)    ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
(p

(train_pypots_model pid=121926) 2025-06-07 15:13:48 [INFO]: No given device, using default device: cpu
(train_pypots_model pid=121926) 2025-06-07 15:13:48 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
(train_pypots_model pid=121926) 2025-06-07 15:13:48 [INFO]: Using customized CrossEntropy as the training loss function.
(train_pypots_model pid=121926) 2025-06-07 15:13:48 [INFO]: Using customized CrossEntropy as the validation metric function.
(pid=121930) 2025-06-07 15:13:41.404204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered [repeated 4x across cluster]
(pid=121927) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR [repeated 4x across cluster]
(pid=121927) E0000 00:00:1749309221.526393  121927 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register fac


Trial train_pypots_model_fb786_00002 started with configuration:
+---------------------------------------------------------+
| Trial train_pypots_model_fb786_00002 config             |
+---------------------------------------------------------+
| d_ffn                                               256 |
| d_model                                              56 |
| dropout                                             0.3 |
| lr                                              0.00134 |
| n_heads                                               1 |
| n_layers                                              2 |
+---------------------------------------------------------+
(pid=121920) 
(pid=121920) 

Trial train_pypots_model_fb786_00001 started with configuration:
+---------------------------------------------------------+
| Trial train_pypots_model_fb786_00001 config             |
+---------------------------------------------------------+
| d_ffn                                                64 |


...


Trial train_pypots_model_fb786_00000 completed after 10 iterations at 2025-06-07 15:15:09. Total running time: 1min 34s
+-------------------------------------------------------------------+
| Trial train_pypots_model_fb786_00000 result                       |
+-------------------------------------------------------------------+
| checkpoint_dir_name                             checkpoint_000009 |
| time_this_iter_s                                          7.37039 |
| time_total_s                                             79.22917 |
| training_iteration                                             10 |
| accuracy                                                  0.91515 |
| f1                                                        0.90841 |
| precision                                                 0.94369 |
| recall                                                    0.87568 |
+-------------------------------------------------------------------+


(train_pypots_model pid=121930) 2025-06-07 15:15:14 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.1805, validation CrossEntropy: 0.1765 [repeated 3x across cluster]
(train_pypots_model pid=121930) 2025-06-07 15:15:14 [INFO]: Finished training. The best model is from epoch#1. [repeated 3x across cluster]
(train_pypots_model pid=121920) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_pypots_model_2025-06-07_15-13-35/train_pypots_model_fb786_00002_2_d_ffn=256,d_model=56,dropout=0.3000,lr=0.0013,n_heads=1,n_layers=2_2025-06-07_15-13-35/checkpoint_000008) [repeated 3x across cluster]
(train_pypots_model pid=121920) 2025-06-07 15:15:10 [INFO]: Saved the model to /tmp/tmpfr1fbtk4/checkpoint.pypots [repeated 3x across cluster]



Trial train_pypots_model_fb786_00003 completed after 10 iterations at 2025-06-07 15:15:15. Total running time: 1min 39s
+-------------------------------------------------------------------+
| Trial train_pypots_model_fb786_00003 result                       |
+-------------------------------------------------------------------+
| checkpoint_dir_name                             checkpoint_000009 |
| time_this_iter_s                                          7.66751 |
| time_total_s                                             84.52142 |
| training_iteration                                             10 |
| accuracy                                                  0.94113 |
| f1                                                        0.93784 |
| precision                                                 0.95176 |
| recall                                                    0.92432 |
+-------------------------------------------------------------------+


2025-06-07 15:15:19,156	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_pypots_model_2025-06-07_15-13-35' in 0.0055s.



Trial train_pypots_model_fb786_00002 completed after 10 iterations at 2025-06-07 15:15:19. Total running time: 1min 43s
+-------------------------------------------------------------------+
| Trial train_pypots_model_fb786_00002 result                       |
+-------------------------------------------------------------------+
| checkpoint_dir_name                             checkpoint_000009 |
| time_this_iter_s                                          8.40963 |
| time_total_s                                              89.7977 |
| training_iteration                                             10 |
| accuracy                                                   0.9316 |
| f1                                                        0.92554 |
| precision                                                 0.97036 |
| recall                                                    0.88468 |
+-------------------------------------------------------------------+

Trial status: 5 TERMINATED
Current tim

## Raindrop

In [84]:
scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=50,
    grace_period=2,
    reduction_factor=2
)

model_base_params={
  "n_steps": X_train_bal.shape[1],
  "n_features": X_train_bal.shape[2],
  "n_classes": len(np.unique(y_train_bal)),
  "batch_size": 64,
  # "patience": 6,
  "num_workers": 0,
  "device": None,
  "model_saving_strategy": None,
  "verbose": True
}

tuner = tune.Tuner(
    tune.with_resources(
      tune.with_parameters(
          train_pypots_model,
          model_cls=Raindrop,
          model_base_params=model_base_params,
          X_train=X_train_bal,
          y_train=y_train_bal,
          X_val=X_val_bal,
          y_val=y_val_bal,
      ),
      resources={"cpu": 1}
    ),
    param_space={
        "n_layers": tune.grid_search([1, 2]),
        "d_ffn": tune.grid_search([32, 64, 128, 256]),
        "n_heads": tune.grid_search([1, 2]),
        "d_model": tune.grid_search([X_train_bal.shape[2] * i for i in range(1, 4+1)]),
        "dropout": tune.grid_search([0.1, 0.2, 0.3]),
        "lr": tune.grid_search([1e-4, 1e-3, 1e-2]),
    },
    tune_config=tune.TuneConfig(
        # num_samples=3,
        metric="accuracy",
        mode="max",
        scheduler=scheduler,
    )
)

results = tuner.fit()
best_results = results.get_best_result(metric='accuracy', mode='max')
best_metrics = best_results.metrics
print(f"Best params: {best_results.config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

Output hidden; open in https://colab.research.google.com to view.

In [86]:
best_results.config

{'n_layers': 1, 'd_ffn': 64, 'n_heads': 1, 'd_model': 56, 'dropout': 0.1}

In [90]:
# lr = best_results.config['lr']
config = best_results.config.copy()
# del config['lr']


run_model(
  lambda: Raindrop(
      **config,
      **model_base_params,
      epochs=50,
      optimizer=Adam(lr=1e-3),
  ),
  f"Raindrop",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 17:19:32 [INFO]: No given device, using default device: cpu
2025-06-07 17:19:32 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-06-07 17:19:32 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 17:19:32 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_uniform also known as glorot
2025-06-07 17:19:32 [INFO]: Raindrop initialized with the given hyperparameters, the number of trainabl

Raindrop & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[342.13278317451477]
342.1328 ± 0.0000


2025-06-07 17:25:24 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.4204, validation CrossEntropy: 0.2247
2025-06-07 17:25:31 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2404, validation CrossEntropy: 0.2922
2025-06-07 17:25:38 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2556, validation CrossEntropy: 0.2064
2025-06-07 17:25:44 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2084, validation CrossEntropy: 0.1927
2025-06-07 17:25:51 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1912, validation CrossEntropy: 0.2126
2025-06-07 17:25:57 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1734, validation CrossEntropy: 0.2185
2025-06-07 17:26:04 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1657, validation CrossEntropy: 0.1870
2025-06-07 17:26:11 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1625, validation CrossEntropy: 0.1656
2025-06-07 17:26:18 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1530, validation CrossEntropy: 0.2074
2025-06-07

Raindrop & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[342.13278317451477, 330.7858281135559]
336.4593 ± 5.6735


2025-06-07 17:31:00 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3731, validation CrossEntropy: 0.2221
2025-06-07 17:31:06 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2327, validation CrossEntropy: 0.2033
2025-06-07 17:31:13 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.1998, validation CrossEntropy: 0.1981
2025-06-07 17:31:19 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.1903, validation CrossEntropy: 0.2558
2025-06-07 17:31:26 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1798, validation CrossEntropy: 0.1773
2025-06-07 17:31:33 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1823, validation CrossEntropy: 0.2733
2025-06-07 17:31:39 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1969, validation CrossEntropy: 0.1809
2025-06-07 17:31:46 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1699, validation CrossEntropy: 0.2015
2025-06-07 17:31:52 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1672, validation CrossEntropy: 0.1759
2025-06-07

...

2025-06-07 17:36:33 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3233, validation CrossEntropy: 0.2643
2025-06-07 17:36:40 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2020, validation CrossEntropy: 0.1735
2025-06-07 17:36:46 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.1994, validation CrossEntropy: 0.2066
2025-06-07 17:36:53 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.1751, validation CrossEntropy: 0.2063
2025-06-07 17:36:59 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1606, validation CrossEntropy: 0.1510
2025-06-07 17:37:06 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1525, validation CrossEntropy: 0.1861
2025-06-07 17:37:12 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1497, validation CrossEntropy: 0.1514
2025-06-07 17:37:19 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1579, validation CrossEntropy: 0.1782
2025-06-07 17:37:26 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1567, validation CrossEntropy: 0.2104
2025-06-07

Raindrop & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[342.13278317451477, 330.7858281135559, 330.0723702907562, 332.1219642162323]
333.7782 ± 4.8793


2025-06-07 17:42:09 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.4179, validation CrossEntropy: 0.2110
2025-06-07 17:42:16 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2907, validation CrossEntropy: 0.2415
2025-06-07 17:42:23 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2419, validation CrossEntropy: 0.3836
2025-06-07 17:42:29 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2222, validation CrossEntropy: 0.2311
2025-06-07 17:42:36 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2003, validation CrossEntropy: 0.2079
2025-06-07 17:42:42 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1896, validation CrossEntropy: 0.2228
2025-06-07 17:42:49 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1836, validation CrossEntropy: 0.1932
2025-06-07 17:42:56 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1903, validation CrossEntropy: 0.1709
2025-06-07 17:43:02 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1862, validation CrossEntropy: 0.1985
2025-06-07

Raindrop & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[342.13278317451477, 330.7858281135559, 330.0723702907562, 332.1219642162323, 332.2475962638855]
333.4721 ± 4.4069


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [91]:
scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=10,
    grace_period=1,
    reduction_factor=3
)

model_base_params={
  "n_steps": X_train_bal.shape[1],
  "n_features": X_train_bal.shape[2],
  "n_classes": len(np.unique(y_train_bal)),
  "batch_size": 64,
  # "patience": 6,
  "num_workers": 0,
  "device": None,
  "model_saving_strategy": None,
  "verbose": True
}

tuner = tune.Tuner(
    tune.with_resources(
      tune.with_parameters(
          train_pypots_model,
          model_cls=Raindrop,
          model_base_params=model_base_params,
          X_train=X_train_bal,
          y_train=y_train_bal,
          X_val=X_val_bal,
          y_val=y_val_bal,
      ),
      resources={"cpu": 1}
    ),
    param_space={
        "n_layers": tune.grid_search([1, 2]),
        "d_ffn": tune.grid_search([32, 64, 128, 256]),
        "n_heads": tune.grid_search([1, 2]),
        "d_model": tune.grid_search([X_train_bal.shape[2] * i for i in range(1, 4+1)]),
        "dropout": tune.grid_search([0.1, 0.2, 0.3]),
        "lr": tune.grid_search([1e-4, 1e-3, 1e-2]),
    },
    tune_config=tune.TuneConfig(
        # num_samples=3,
        metric="f1",
        mode="max",
        scheduler=scheduler,
    )
)

results = tuner.fit()
best_results = results.get_best_result(metric='f1', mode='max')
best_metrics = best_results.metrics
print(f"Best params: {best_results.config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

Output hidden; open in https://colab.research.google.com to view.

In [92]:
# lr = best_results.config['lr']
config = best_results.config.copy()
# del config['lr']


run_model(
  lambda: Raindrop(
      **config,
      **model_base_params,
      epochs=50,
      optimizer=Adam(lr=1e-3),
  ),
  f"Raindrop",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 19:42:15 [INFO]: No given device, using default device: cpu
2025-06-07 19:42:15 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-06-07 19:42:15 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 19:42:15 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_uniform also known as glorot
2025-06-07 19:42:15 [INFO]: Raindrop initialized with the given hyperparameters, the number of trainabl

Raindrop & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[381.01662278175354]
381.0166 ± 0.0000


2025-06-07 19:48:48 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3960, validation CrossEntropy: 0.2456
2025-06-07 19:48:55 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2204, validation CrossEntropy: 0.1919
2025-06-07 19:49:06 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.1942, validation CrossEntropy: 0.2453
2025-06-07 19:49:13 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.1813, validation CrossEntropy: 0.2480
2025-06-07 19:49:21 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1748, validation CrossEntropy: 0.1835
2025-06-07 19:49:29 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2041, validation CrossEntropy: 0.1823
2025-06-07 19:49:36 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1924, validation CrossEntropy: 0.2044
2025-06-07 19:49:44 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1717, validation CrossEntropy: 0.1947
2025-06-07 19:49:51 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1807, validation CrossEntropy: 0.1921
2025-06-07

Raindrop & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[381.01662278175354, 380.8622155189514]
380.9394 ± 0.0772


2025-06-07 19:55:12 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.4307, validation CrossEntropy: 0.2664
2025-06-07 19:55:20 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2673, validation CrossEntropy: 0.2712
2025-06-07 19:55:27 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2377, validation CrossEntropy: 0.2278
2025-06-07 19:55:35 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2350, validation CrossEntropy: 0.2180
2025-06-07 19:55:42 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2161, validation CrossEntropy: 0.2401
2025-06-07 19:55:49 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1921, validation CrossEntropy: 0.2038
2025-06-07 19:55:57 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1946, validation CrossEntropy: 0.2039
2025-06-07 19:56:04 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1755, validation CrossEntropy: 0.2382
2025-06-07 19:56:11 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1695, validation CrossEntropy: 0.2557
2025-06-07

Raindrop & 0.4881 ± 0.0173 & 0.6504 ± 0.0077 0.4819 ± 0.0085 1.0000 ± 0.0000 \
[381.01662278175354, 380.8622155189514, 377.30623388290405]
379.7284 ± 1.7139


2025-06-07 20:01:36 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3813, validation CrossEntropy: 0.2265
2025-06-07 20:01:44 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2462, validation CrossEntropy: 0.1970
2025-06-07 20:01:53 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2034, validation CrossEntropy: 0.2336
2025-06-07 20:02:01 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.1823, validation CrossEntropy: 0.2271
2025-06-07 20:02:09 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1719, validation CrossEntropy: 0.2118
2025-06-07 20:02:18 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1614, validation CrossEntropy: 0.1637
2025-06-07 20:02:25 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1583, validation CrossEntropy: 0.1611
2025-06-07 20:02:33 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1541, validation CrossEntropy: 0.1986
2025-06-07 20:02:41 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1488, validation CrossEntropy: 0.1860
2025-06-07

Raindrop & 0.5871 ± 0.1720 & 0.7099 ± 0.1034 0.5658 ± 0.1454 0.9934 ± 0.0114 \
[381.01662278175354, 380.8622155189514, 377.30623388290405, 394.3478653430939]
383.3832 ± 6.5021


2025-06-07 20:08:13 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.4197, validation CrossEntropy: 0.2351
2025-06-07 20:08:21 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2737, validation CrossEntropy: 0.3362
2025-06-07 20:08:28 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2240, validation CrossEntropy: 0.2077
2025-06-07 20:08:36 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2111, validation CrossEntropy: 0.2144
2025-06-07 20:08:44 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1954, validation CrossEntropy: 0.2043
2025-06-07 20:08:52 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1800, validation CrossEntropy: 0.2039
2025-06-07 20:09:00 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1721, validation CrossEntropy: 0.2553
2025-06-07 20:09:08 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1650, validation CrossEntropy: 0.2285
2025-06-07 20:09:16 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1608, validation CrossEntropy: 0.1808
2025-06-07

Raindrop & 0.6541 ± 0.2041 & 0.7523 ± 0.1255 0.6297 ± 0.1824 0.9870 ± 0.0164 \
[381.01662278175354, 380.8622155189514, 377.30623388290405, 394.3478653430939, 392.31759905815125]
385.1701 ± 6.8259


In [ ]:
n_layers=2
d_ffn=256
n_heads=2
dropout=0.3
run_model(
  lambda: Raindrop(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    n_layers=2,
    d_model=X_train_bal.shape[2] * 4,
    d_ffn=256,
    n_heads=2,
    dropout=0.3,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/raindrop',
    model_saving_strategy='best'
  ),
  f"Raindrop({n_layers=}, {d_ffn=}, {n_heads=}, {dropout=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-06 22:52:12 [INFO]: No given device, using default device: cpu
2025-06-06 22:52:12 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250606_T225212
2025-06-06 22:52:12 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250606_T225212/tensorboard
2025-06-06 22:52:12 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-06 22:52:12 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_

Raindrop(n_layers=2, d_ffn=256, n_heads=2, dropout=0.3) & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[258.91557335853577]
258.9156 ± 0.0000


2025-06-06 22:56:48 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.5044, validation CrossEntropy: 0.3056
2025-06-06 22:57:01 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.3690, validation CrossEntropy: 0.1805
2025-06-06 22:57:12 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.3077, validation CrossEntropy: 0.1848
2025-06-06 22:57:22 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2851, validation CrossEntropy: 0.2517
2025-06-06 22:57:33 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2964, validation CrossEntropy: 0.2272
2025-06-06 22:57:44 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2768, validation CrossEntropy: 0.1798
2025-06-06 22:57:55 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.2553, validation CrossEntropy: 0.2230
2025-06-06 22:58:06 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.2623, validation CrossEntropy: 0.1781
2025-06-06 22:58:17 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.2528, validation CrossEntropy: 0.2680
2025-06-06

Raindrop(n_layers=2, d_ffn=256, n_heads=2, dropout=0.3) & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[258.91557335853577, 329.2944755554199]
294.1050 ± 35.1895


2025-06-06 23:02:23 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.5433, validation CrossEntropy: 0.2829
2025-06-06 23:02:34 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.3624, validation CrossEntropy: 0.1896
2025-06-06 23:02:45 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.3049, validation CrossEntropy: 0.3274
2025-06-06 23:02:56 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.3028, validation CrossEntropy: 0.2400
2025-06-06 23:03:07 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2821, validation CrossEntropy: 0.2372
2025-06-06 23:03:19 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2666, validation CrossEntropy: 0.2814
2025-06-06 23:03:30 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.2636, validation CrossEntropy: 0.2108
2025-06-06 23:03:41 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.2656, validation CrossEntropy: 0.1527
2025-06-06 23:03:52 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.2521, validation CrossEntropy: 0.2264
2025-06-06

...

2025-06-06 23:07:35 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.5006, validation CrossEntropy: 0.2221
2025-06-06 23:07:46 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.3603, validation CrossEntropy: 0.1898
2025-06-06 23:07:57 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.3259, validation CrossEntropy: 0.1599
2025-06-06 23:08:07 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2972, validation CrossEntropy: 0.1539
2025-06-06 23:08:17 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2822, validation CrossEntropy: 0.1863
2025-06-06 23:08:28 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2559, validation CrossEntropy: 0.1461
2025-06-06 23:08:39 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.2531, validation CrossEntropy: 0.1383
2025-06-06 23:08:51 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.2445, validation CrossEntropy: 0.1314
2025-06-06 23:09:01 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.2344, validation CrossEntropy: 0.1338
2025-06-06

Raindrop(n_layers=2, d_ffn=256, n_heads=2, dropout=0.3) & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[258.91557335853577, 329.2944755554199, 304.7034709453583, 333.84255361557007]
306.6890 ± 29.7261


2025-06-06 23:13:14 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.4941, validation CrossEntropy: 0.2442
2025-06-06 23:13:25 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.3463, validation CrossEntropy: 0.1769
2025-06-06 23:13:37 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.3264, validation CrossEntropy: 0.2524
2025-06-06 23:13:47 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2964, validation CrossEntropy: 0.1703
2025-06-06 23:13:58 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2952, validation CrossEntropy: 0.1625
2025-06-06 23:14:09 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2609, validation CrossEntropy: 0.1596
2025-06-06 23:14:20 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.2553, validation CrossEntropy: 0.1646
2025-06-06 23:14:31 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.2480, validation CrossEntropy: 0.1392
2025-06-06 23:14:41 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.2375, validation CrossEntropy: 0.1609
2025-06-06

Raindrop(n_layers=2, d_ffn=256, n_heads=2, dropout=0.3) & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[258.91557335853577, 329.2944755554199, 304.7034709453583, 333.84255361557007, 211.1162383556366]
287.5745 ± 46.5658


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
n_layers=2
d_ffn=256
n_heads=2
dropout=0.1
d_model = X_train_bal.shape[2] * 4
run_model(
  lambda: Raindrop(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    n_layers=n_layers,
    d_model=X_train_bal.shape[2] * 4,
    d_ffn=d_ffn,
    n_heads=n_heads,
    dropout=dropout,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/raindrop',
    model_saving_strategy='best'
  ),
  f"Raindrop({n_layers=}, {d_ffn=}, {d_model=}, {n_heads=}, {dropout=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-06 23:16:38 [INFO]: No given device, using default device: cpu
2025-06-06 23:16:38 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250606_T231638
2025-06-06 23:16:38 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250606_T231638/tensorboard
2025-06-06 23:16:38 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-06 23:16:38 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_

Raindrop(n_layers=2, d_ffn=256, d_model=56, n_heads=2, dropout=0.1) & 0.5569 ± 0.0000 & 0.6823 ± 0.0000 0.5178 ± 0.0000 1.0000 ± 0.0000 \
[330.12748312950134]
330.1275 ± 0.0000


2025-06-06 23:22:26 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.4396, validation CrossEntropy: 0.2302
2025-06-06 23:22:37 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2929, validation CrossEntropy: 0.2321
2025-06-06 23:22:48 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2450, validation CrossEntropy: 0.1591
2025-06-06 23:22:59 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2419, validation CrossEntropy: 0.1666
2025-06-06 23:23:09 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2278, validation CrossEntropy: 0.1811
2025-06-06 23:23:21 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2140, validation CrossEntropy: 0.1889
2025-06-06 23:23:32 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.2020, validation CrossEntropy: 0.1517
2025-06-06 23:23:44 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1919, validation CrossEntropy: 0.1648
2025-06-06 23:23:54 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1845, validation CrossEntropy: 0.1574
2025-06-06

Raindrop(n_layers=2, d_ffn=256, d_model=56, n_heads=2, dropout=0.1) & 0.5164 ± 0.0405 & 0.6636 ± 0.0187 0.4969 ± 0.0210 1.0000 ± 0.0000 \
[330.12748312950134, 229.8338565826416]
279.9807 ± 50.1468


2025-06-06 23:26:22 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3723, validation CrossEntropy: 0.1754
2025-06-06 23:26:32 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2448, validation CrossEntropy: 0.1938
2025-06-06 23:26:43 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2251, validation CrossEntropy: 0.1672
2025-06-06 23:26:54 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.1955, validation CrossEntropy: 0.2383
2025-06-06 23:27:04 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2077, validation CrossEntropy: 0.1486
2025-06-06 23:27:15 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1941, validation CrossEntropy: 0.1873
2025-06-06 23:27:26 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1884, validation CrossEntropy: 0.1600
2025-06-06 23:27:38 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1911, validation CrossEntropy: 0.1554
2025-06-06 23:27:49 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1916, validation CrossEntropy: 0.1657
2025-06-06

...

2025-06-06 23:30:17 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.2898, validation CrossEntropy: 0.1328
2025-06-06 23:30:28 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.1906, validation CrossEntropy: 0.1227
2025-06-06 23:30:38 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.1705, validation CrossEntropy: 0.1166
2025-06-06 23:30:49 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.1643, validation CrossEntropy: 0.1110
2025-06-06 23:30:59 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1555, validation CrossEntropy: 0.1251
2025-06-06 23:31:10 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1460, validation CrossEntropy: 0.1172
2025-06-06 23:31:20 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1493, validation CrossEntropy: 0.1153
2025-06-06 23:31:31 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1391, validation CrossEntropy: 0.1152
2025-06-06 23:31:42 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1393, validation CrossEntropy: 0.1190
2025-06-06

Raindrop(n_layers=2, d_ffn=256, d_model=56, n_heads=2, dropout=0.1) & 0.5004 ± 0.0333 & 0.6561 ± 0.0154 0.4884 ± 0.0173 1.0000 ± 0.0000 \
[330.12748312950134, 229.8338565826416, 230.84162664413452, 298.7073378562927]
272.3776 ± 43.4842


2025-06-06 23:35:22 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3992, validation CrossEntropy: 0.2408
2025-06-06 23:35:32 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2639, validation CrossEntropy: 0.1862
2025-06-06 23:35:44 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2308, validation CrossEntropy: 0.1642
2025-06-06 23:35:55 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2250, validation CrossEntropy: 0.1429
2025-06-06 23:36:06 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2316, validation CrossEntropy: 0.1362
2025-06-06 23:36:16 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2160, validation CrossEntropy: 0.1429
2025-06-06 23:36:26 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.2155, validation CrossEntropy: 0.1587
2025-06-06 23:36:37 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1914, validation CrossEntropy: 0.1515
2025-06-06 23:36:48 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1847, validation CrossEntropy: 0.1401
2025-06-06

Raindrop(n_layers=2, d_ffn=256, d_model=56, n_heads=2, dropout=0.1) & 0.4955 ± 0.0314 & 0.6539 ± 0.0145 0.4859 ± 0.0163 1.0000 ± 0.0000 \
[330.12748312950134, 229.8338565826416, 230.84162664413452, 298.7073378562927, 119.19620251655579]
241.7413 ± 72.5743


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
n_layers=1
d_ffn=128
n_heads=2
dropout=0.1
d_model = X_train_bal.shape[2] * 2
run_model(
  lambda: Raindrop(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    n_layers=n_layers,
    d_model=X_train_bal.shape[2] * 4,
    d_ffn=d_ffn,
    n_heads=n_heads,
    dropout=dropout,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/raindrop',
    model_saving_strategy='best'
  ),
  f"Raindrop({n_layers=}, {d_ffn=}, {d_model=}, {n_heads=}, {dropout=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-06 23:37:15 [INFO]: No given device, using default device: cpu
2025-06-06 23:37:15 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250606_T233715
2025-06-06 23:37:15 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250606_T233715/tensorboard
2025-06-06 23:37:15 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-06 23:37:15 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_

Raindrop(n_layers=1, d_ffn=128, d_model=28, n_heads=2, dropout=0.1) & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[303.4940321445465]
303.4940 ± 0.0000


2025-06-06 23:42:35 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3936, validation CrossEntropy: 0.1842
2025-06-06 23:42:45 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2596, validation CrossEntropy: 0.1953
2025-06-06 23:42:54 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2294, validation CrossEntropy: 0.1891
2025-06-06 23:43:05 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2196, validation CrossEntropy: 0.1383
2025-06-06 23:43:15 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2034, validation CrossEntropy: 0.1641
2025-06-06 23:43:25 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1900, validation CrossEntropy: 0.1401
2025-06-06 23:43:34 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.2140, validation CrossEntropy: 0.1725
2025-06-06 23:43:46 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.2043, validation CrossEntropy: 0.1339
2025-06-06 23:43:56 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.2059, validation CrossEntropy: 0.1978
2025-06-06

Raindrop(n_layers=1, d_ffn=128, d_model=28, n_heads=2, dropout=0.1) & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[303.4940321445465, 305.7749571800232]
304.6345 ± 1.1405


2025-06-06 23:47:48 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.4395, validation CrossEntropy: 0.1929
2025-06-06 23:47:57 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2897, validation CrossEntropy: 0.2600
2025-06-06 23:48:07 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2618, validation CrossEntropy: 0.1709
2025-06-06 23:48:18 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2475, validation CrossEntropy: 0.1795
2025-06-06 23:48:27 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2259, validation CrossEntropy: 0.1834
2025-06-06 23:48:37 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2070, validation CrossEntropy: 0.1411
2025-06-06 23:48:47 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.2101, validation CrossEntropy: 0.1420
2025-06-06 23:48:57 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.2015, validation CrossEntropy: 0.1678
2025-06-06 23:49:06 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1927, validation CrossEntropy: 0.1813
2025-06-06

...

2025-06-06 23:49:55 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3833, validation CrossEntropy: 0.1767
2025-06-06 23:50:05 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2470, validation CrossEntropy: 0.1816
2025-06-06 23:50:14 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2231, validation CrossEntropy: 0.1456
2025-06-06 23:50:24 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2047, validation CrossEntropy: 0.1387
2025-06-06 23:50:35 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1974, validation CrossEntropy: 0.1351
2025-06-06 23:50:45 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2100, validation CrossEntropy: 0.1484
2025-06-06 23:50:55 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1876, validation CrossEntropy: 0.1376
2025-06-06 23:51:06 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1787, validation CrossEntropy: 0.1314
2025-06-06 23:51:16 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1726, validation CrossEntropy: 0.1553
2025-06-06

Raindrop(n_layers=1, d_ffn=128, d_model=28, n_heads=2, dropout=0.1) & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[303.4940321445465, 305.7749571800232, 121.09035158157349, 176.25129508972168]
226.6527 ± 80.3876


2025-06-06 23:52:56 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.4023, validation CrossEntropy: 0.1882
2025-06-06 23:53:06 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2568, validation CrossEntropy: 0.2140
2025-06-06 23:53:15 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2326, validation CrossEntropy: 0.1684
2025-06-06 23:53:25 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.1983, validation CrossEntropy: 0.1519
2025-06-06 23:53:35 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1885, validation CrossEntropy: 0.1445
2025-06-06 23:53:46 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1826, validation CrossEntropy: 0.1343
2025-06-06 23:53:56 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1802, validation CrossEntropy: 0.1819
2025-06-06 23:54:06 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1656, validation CrossEntropy: 0.1265
2025-06-06 23:54:16 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1579, validation CrossEntropy: 0.1550
2025-06-06

Raindrop(n_layers=1, d_ffn=128, d_model=28, n_heads=2, dropout=0.1) & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[303.4940321445465, 305.7749571800232, 121.09035158157349, 176.25129508972168, 253.68429231643677]
232.0590 ± 72.7093


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
n_layers=2
d_ffn=32
n_heads=3
dropout=0.1
d_model = X_train_bal.shape[2] * 1
run_model(
  lambda: Raindrop(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    n_layers=n_layers,
    d_model=X_train_bal.shape[2] * 4,
    d_ffn=d_ffn,
    n_heads=n_heads,
    dropout=dropout,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/raindrop',
    model_saving_strategy='best'
  ),
  f"Raindrop({n_layers=}, {d_ffn=}, {d_model=}, {n_heads=}, {dropout=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-06 23:57:03 [INFO]: No given device, using default device: cpu
2025-06-06 23:57:03 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250606_T235703
2025-06-06 23:57:03 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250606_T235703/tensorboard
2025-06-06 23:57:03 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-06 23:57:03 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_

Raindrop(n_layers=2, d_ffn=32, d_model=14, n_heads=3, dropout=0.1) & 0.4759 ± 0.0000 & 0.6449 ± 0.0000 0.4759 ± 0.0000 1.0000 ± 0.0000 \
[107.2324903011322]
107.2325 ± 0.0000


2025-06-06 23:59:07 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3959, validation CrossEntropy: 0.3169
2025-06-06 23:59:18 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2612, validation CrossEntropy: 0.1542
2025-06-06 23:59:28 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2482, validation CrossEntropy: 0.1580
2025-06-06 23:59:37 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2130, validation CrossEntropy: 0.2154
2025-06-06 23:59:49 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2031, validation CrossEntropy: 0.1481
2025-06-07 00:00:00 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1984, validation CrossEntropy: 0.1595
2025-06-07 00:00:10 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1768, validation CrossEntropy: 0.1402
2025-06-07 00:00:20 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1713, validation CrossEntropy: 0.1321
2025-06-07 00:00:30 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1651, validation CrossEntropy: 0.1449
2025-06-07

Raindrop(n_layers=2, d_ffn=32, d_model=14, n_heads=3, dropout=0.1) & 0.4781 ± 0.0022 & 0.6459 ± 0.0010 0.4770 ± 0.0011 1.0000 ± 0.0000 \
[107.2324903011322, 166.38590335845947]
136.8092 ± 29.5767


2025-06-07 00:02:00 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3913, validation CrossEntropy: 0.1791
2025-06-07 00:02:10 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2012, validation CrossEntropy: 0.1325
2025-06-07 00:02:20 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.1837, validation CrossEntropy: 0.1299
2025-06-07 00:02:31 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.1768, validation CrossEntropy: 0.1165
2025-06-07 00:02:41 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1729, validation CrossEntropy: 0.1305
2025-06-07 00:02:51 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1770, validation CrossEntropy: 0.1131
2025-06-07 00:03:01 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1626, validation CrossEntropy: 0.1189
2025-06-07 00:03:12 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1581, validation CrossEntropy: 0.1157
2025-06-07 00:03:22 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1498, validation CrossEntropy: 0.1594
2025-06-07

Raindrop(n_layers=2, d_ffn=32, d_model=14, n_heads=3, dropout=0.1) & 0.4776 ± 0.0020 & 0.6456 ± 0.0009 0.4767 ± 0.0010 1.0000 ± 0.0000 \
[107.2324903011322, 166.38590335845947, 206.32819938659668]
159.9822 ± 40.7083


2025-06-07 00:05:32 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3727, validation CrossEntropy: 0.1865
2025-06-07 00:05:43 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2446, validation CrossEntropy: 0.1534
2025-06-07 00:05:54 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2103, validation CrossEntropy: 0.1849
2025-06-07 00:06:04 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2075, validation CrossEntropy: 0.1509
2025-06-07 00:06:14 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.1934, validation CrossEntropy: 0.1407
2025-06-07 00:06:25 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.1858, validation CrossEntropy: 0.1390
2025-06-07 00:06:36 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.1994, validation CrossEntropy: 0.1692
2025-06-07 00:06:45 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.1889, validation CrossEntropy: 0.1592
2025-06-07 00:06:55 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.1853, validation CrossEntropy: 0.1614
2025-06-07

Raindrop(n_layers=2, d_ffn=32, d_model=14, n_heads=3, dropout=0.1) & 0.4771 ± 0.0019 & 0.6454 ± 0.0008 0.4765 ± 0.0009 1.0000 ± 0.0000 \
[107.2324903011322, 166.38590335845947, 206.32819938659668, 230.85198497772217]
177.6996 ± 46.7397


2025-06-07 00:09:28 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.3641, validation CrossEntropy: 0.1861
2025-06-07 00:09:38 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.2658, validation CrossEntropy: 0.1687
2025-06-07 00:09:50 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.2362, validation CrossEntropy: 0.1768
2025-06-07 00:10:00 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.2345, validation CrossEntropy: 0.1814
2025-06-07 00:10:10 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.2183, validation CrossEntropy: 0.1511
2025-06-07 00:10:21 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.2164, validation CrossEntropy: 0.1644
2025-06-07 00:10:31 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.2000, validation CrossEntropy: 0.1577
2025-06-07 00:10:41 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.2116, validation CrossEntropy: 0.2250
2025-06-07 00:10:51 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.2058, validation CrossEntropy: 0.1448
2025-06-07

Raindrop(n_layers=2, d_ffn=32, d_model=14, n_heads=3, dropout=0.1) & 0.5407 ± 0.1271 & 0.6806 ± 0.0704 0.5214 ± 0.0899 0.9982 ± 0.0035 \
[107.2324903011322, 166.38590335845947, 206.32819938659668, 230.85198497772217, 188.99000597000122]
179.9577 ± 42.0485


## Brits

In [ ]:

rnn_hidden_size=64
classification_weight=1
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 00:12:30 [INFO]: No given device, using default device: cpu
2025-06-07 00:12:30 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T001230
2025-06-07 00:12:30 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T001230/tensorboard
2025-06-07 00:12:30 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 00:12:30 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 00:12:30 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 53,780
2025-06-07 00:12:40 [INFO]: Epoch 001 - training loss (CrossEntropy): 2536.8028, validation CrossEntropy: 0.4211
2025-06-07 00:12:45 [INFO]: Epoch 002 - training loss (CrossEntropy): 2009.3527, validation CrossEntropy: 0.3393
2025-06-07 00:12:50 [INFO]: Epoch 003 - training loss (CrossEntropy): 1730.4002, validation CrossEntropy: 0.3509
2025-06-07 00:12:55 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=1) & 0.5241 ± 0.0000 & 0.0000 ± 0.0000 0.0000 ± 0.0000 0.0000 ± 0.0000 \
[155.95759844779968]
155.9576 ± 0.0000


2025-06-07 00:15:19 [INFO]: Epoch 001 - training loss (CrossEntropy): 2710.6285, validation CrossEntropy: 0.4120
2025-06-07 00:15:23 [INFO]: Epoch 002 - training loss (CrossEntropy): 2156.0005, validation CrossEntropy: 0.3723
2025-06-07 00:15:28 [INFO]: Epoch 003 - training loss (CrossEntropy): 1863.6658, validation CrossEntropy: 0.3722
2025-06-07 00:15:34 [INFO]: Epoch 004 - training loss (CrossEntropy): 1691.4819, validation CrossEntropy: 0.3368
2025-06-07 00:15:39 [INFO]: Epoch 005 - training loss (CrossEntropy): 1598.7497, validation CrossEntropy: 0.3095
2025-06-07 00:15:45 [INFO]: Epoch 006 - training loss (CrossEntropy): 1584.5198, validation CrossEntropy: 0.2961
2025-06-07 00:15:50 [INFO]: Epoch 007 - training loss (CrossEntropy): 1577.6814, validation CrossEntropy: 0.2694
2025-06-07 00:15:55 [INFO]: Epoch 008 - training loss (CrossEntropy): 1573.3751, validation CrossEntropy: 0.2583
2025-06-07 00:16:00 [INFO]: Epoch 009 - training loss (CrossEntropy): 1569.7682, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=1) & 0.6116 ± 0.0875 & 0.2840 ± 0.2840 0.4483 ± 0.4483 0.2079 ± 0.2079 \
[155.95759844779968, 158.3724308013916]
157.1650 ± 1.2074


2025-06-07 00:18:00 [INFO]: Epoch 001 - training loss (CrossEntropy): 2625.3120, validation CrossEntropy: 0.4063
2025-06-07 00:18:06 [INFO]: Epoch 002 - training loss (CrossEntropy): 2104.6043, validation CrossEntropy: 0.3316
2025-06-07 00:18:10 [INFO]: Epoch 003 - training loss (CrossEntropy): 1828.2200, validation CrossEntropy: 0.3833
2025-06-07 00:18:15 [INFO]: Epoch 004 - training loss (CrossEntropy): 1668.4745, validation CrossEntropy: 0.3410
2025-06-07 00:18:21 [INFO]: Epoch 005 - training loss (CrossEntropy): 1599.9312, validation CrossEntropy: 0.2973
2025-06-07 00:18:25 [INFO]: Epoch 006 - training loss (CrossEntropy): 1583.8307, validation CrossEntropy: 0.2812
2025-06-07 00:18:31 [INFO]: Epoch 007 - training loss (CrossEntropy): 1581.6523, validation CrossEntropy: 0.2662
2025-06-07 00:18:35 [INFO]: Epoch 008 - training loss (CrossEntropy): 1576.7455, validation CrossEntropy: 0.2509
2025-06-07 00:18:40 [INFO]: Epoch 009 - training loss (CrossEntropy): 1570.9225, validation Cros

...

2025-06-07 00:20:39 [INFO]: Epoch 001 - training loss (CrossEntropy): 2634.0404, validation CrossEntropy: 0.4125
2025-06-07 00:20:44 [INFO]: Epoch 002 - training loss (CrossEntropy): 2131.6108, validation CrossEntropy: 0.3368
2025-06-07 00:20:50 [INFO]: Epoch 003 - training loss (CrossEntropy): 1854.7645, validation CrossEntropy: 0.3248
2025-06-07 00:20:55 [INFO]: Epoch 004 - training loss (CrossEntropy): 1660.3477, validation CrossEntropy: 0.3227
2025-06-07 00:20:59 [INFO]: Epoch 005 - training loss (CrossEntropy): 1590.0719, validation CrossEntropy: 0.2974
2025-06-07 00:21:05 [INFO]: Epoch 006 - training loss (CrossEntropy): 1581.2140, validation CrossEntropy: 0.2855
2025-06-07 00:21:10 [INFO]: Epoch 007 - training loss (CrossEntropy): 1575.2545, validation CrossEntropy: 0.2475
2025-06-07 00:21:15 [INFO]: Epoch 008 - training loss (CrossEntropy): 1569.3034, validation CrossEntropy: 0.2285
2025-06-07 00:21:20 [INFO]: Epoch 009 - training loss (CrossEntropy): 1566.9740, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=1) & 0.5983 ± 0.0766 & 0.2524 ± 0.2563 0.4456 ± 0.4456 0.1775 ± 0.1826 \
[155.95759844779968, 158.3724308013916, 155.47723269462585, 136.19043016433716]
151.4994 ± 8.9065


2025-06-07 00:22:58 [INFO]: Epoch 001 - training loss (CrossEntropy): 2619.9030, validation CrossEntropy: 0.4418
2025-06-07 00:23:04 [INFO]: Epoch 002 - training loss (CrossEntropy): 2106.0622, validation CrossEntropy: 0.3527
2025-06-07 00:23:09 [INFO]: Epoch 003 - training loss (CrossEntropy): 1804.5229, validation CrossEntropy: 0.3481
2025-06-07 00:23:14 [INFO]: Epoch 004 - training loss (CrossEntropy): 1657.1325, validation CrossEntropy: 0.3266
2025-06-07 00:23:20 [INFO]: Epoch 005 - training loss (CrossEntropy): 1590.7783, validation CrossEntropy: 0.2945
2025-06-07 00:23:24 [INFO]: Epoch 006 - training loss (CrossEntropy): 1582.8623, validation CrossEntropy: 0.2703
2025-06-07 00:23:29 [INFO]: Epoch 007 - training loss (CrossEntropy): 1574.9534, validation CrossEntropy: 0.2456
2025-06-07 00:23:35 [INFO]: Epoch 008 - training loss (CrossEntropy): 1569.1055, validation CrossEntropy: 0.2548
2025-06-07 00:23:40 [INFO]: Epoch 009 - training loss (CrossEntropy): 1565.7214, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=1) & 0.5835 ± 0.0746 & 0.2019 ± 0.2505 0.3564 ± 0.4366 0.1420 ± 0.1781 \
[155.95759844779968, 158.3724308013916, 155.47723269462585, 136.19043016433716, 91.66017889976501]
139.5316 ± 25.2265


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=32
classification_weight=1
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 00:24:24 [INFO]: No given device, using default device: cpu
2025-06-07 00:24:24 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T002424
2025-06-07 00:24:24 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T002424/tensorboard
2025-06-07 00:24:24 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 00:24:24 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 00:24:24 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,540
2025-06-07 00:24:33 [INFO]: Epoch 001 - training loss (CrossEntropy): 2470.6516, validation CrossEntropy: 0.6000
2025-06-07 00:24:38 [INFO]: Epoch 002 - training loss (CrossEntropy): 2025.5432, validation CrossEntropy: 0.5013
2025-06-07 00:24:43 [INFO]: Epoch 003 - training loss (CrossEntropy): 1780.1451, validation CrossEntropy: 0.4571
2025-06-07 00:24:47 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=1) & 0.7400 ± 0.0000 & 0.7081 ± 0.0000 0.7604 ± 0.0000 0.6624 ± 0.0000 \
[143.43686366081238]
143.4369 ± 0.0000


2025-06-07 00:26:59 [INFO]: Epoch 001 - training loss (CrossEntropy): 2453.1343, validation CrossEntropy: 0.5068
2025-06-07 00:27:03 [INFO]: Epoch 002 - training loss (CrossEntropy): 2007.7135, validation CrossEntropy: 0.3988
2025-06-07 00:27:08 [INFO]: Epoch 003 - training loss (CrossEntropy): 1784.6887, validation CrossEntropy: 0.3989
2025-06-07 00:27:12 [INFO]: Epoch 004 - training loss (CrossEntropy): 1654.5249, validation CrossEntropy: 0.3517
2025-06-07 00:27:17 [INFO]: Epoch 005 - training loss (CrossEntropy): 1592.2027, validation CrossEntropy: 0.3563
2025-06-07 00:27:21 [INFO]: Epoch 006 - training loss (CrossEntropy): 1585.5516, validation CrossEntropy: 0.3463
2025-06-07 00:27:26 [INFO]: Epoch 007 - training loss (CrossEntropy): 1579.5153, validation CrossEntropy: 0.3204
2025-06-07 00:27:31 [INFO]: Epoch 008 - training loss (CrossEntropy): 1579.0481, validation CrossEntropy: 0.3211
2025-06-07 00:27:36 [INFO]: Epoch 009 - training loss (CrossEntropy): 1570.4973, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=1) & 0.7203 ± 0.0198 & 0.6539 ± 0.0542 0.7921 ± 0.0317 0.5669 ± 0.0955 \
[143.43686366081238, 100.86256146430969]
122.1497 ± 21.2872


2025-06-07 00:28:43 [INFO]: Epoch 001 - training loss (CrossEntropy): 2292.7036, validation CrossEntropy: 0.5899
2025-06-07 00:28:48 [INFO]: Epoch 002 - training loss (CrossEntropy): 1909.1357, validation CrossEntropy: 0.4779
2025-06-07 00:28:52 [INFO]: Epoch 003 - training loss (CrossEntropy): 1716.5746, validation CrossEntropy: 0.4314
2025-06-07 00:28:57 [INFO]: Epoch 004 - training loss (CrossEntropy): 1624.0064, validation CrossEntropy: 0.4573
2025-06-07 00:29:02 [INFO]: Epoch 005 - training loss (CrossEntropy): 1592.4683, validation CrossEntropy: 0.4236
2025-06-07 00:29:06 [INFO]: Epoch 006 - training loss (CrossEntropy): 1580.8599, validation CrossEntropy: 0.3864
2025-06-07 00:29:10 [INFO]: Epoch 007 - training loss (CrossEntropy): 1577.9462, validation CrossEntropy: 0.3594
2025-06-07 00:29:16 [INFO]: Epoch 008 - training loss (CrossEntropy): 1573.6972, validation CrossEntropy: 0.3835
2025-06-07 00:29:20 [INFO]: Epoch 009 - training loss (CrossEntropy): 1569.4459, validation Cros

...

2025-06-07 00:31:09 [INFO]: Epoch 001 - training loss (CrossEntropy): 2420.7842, validation CrossEntropy: 0.5846
2025-06-07 00:31:13 [INFO]: Epoch 002 - training loss (CrossEntropy): 1950.1461, validation CrossEntropy: 0.4812
2025-06-07 00:31:18 [INFO]: Epoch 003 - training loss (CrossEntropy): 1730.0199, validation CrossEntropy: 0.4486
2025-06-07 00:31:23 [INFO]: Epoch 004 - training loss (CrossEntropy): 1624.2946, validation CrossEntropy: 0.4170
2025-06-07 00:31:27 [INFO]: Epoch 005 - training loss (CrossEntropy): 1590.0137, validation CrossEntropy: 0.4066
2025-06-07 00:31:32 [INFO]: Epoch 006 - training loss (CrossEntropy): 1582.5991, validation CrossEntropy: 0.3619
2025-06-07 00:31:38 [INFO]: Epoch 007 - training loss (CrossEntropy): 1576.9767, validation CrossEntropy: 0.4011
2025-06-07 00:31:42 [INFO]: Epoch 008 - training loss (CrossEntropy): 1574.7771, validation CrossEntropy: 0.3738
2025-06-07 00:31:47 [INFO]: Epoch 009 - training loss (CrossEntropy): 1568.9227, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=1) & 0.6261 ± 0.0953 & 0.3527 ± 0.3057 0.5739 ± 0.3337 0.2974 ± 0.2786 \
[143.43686366081238, 100.86256146430969, 142.1179211139679, 143.1390471458435]
132.3891 ± 18.2084


2025-06-07 00:33:36 [INFO]: Epoch 001 - training loss (CrossEntropy): 2310.0974, validation CrossEntropy: 0.5840
2025-06-07 00:33:41 [INFO]: Epoch 002 - training loss (CrossEntropy): 1904.6643, validation CrossEntropy: 0.4545
2025-06-07 00:33:46 [INFO]: Epoch 003 - training loss (CrossEntropy): 1734.6008, validation CrossEntropy: 0.4132
2025-06-07 00:33:50 [INFO]: Epoch 004 - training loss (CrossEntropy): 1648.3337, validation CrossEntropy: 0.3629
2025-06-07 00:33:55 [INFO]: Epoch 005 - training loss (CrossEntropy): 1599.2849, validation CrossEntropy: 0.3864
2025-06-07 00:33:59 [INFO]: Epoch 006 - training loss (CrossEntropy): 1584.0845, validation CrossEntropy: 0.3315
2025-06-07 00:34:04 [INFO]: Epoch 007 - training loss (CrossEntropy): 1577.1509, validation CrossEntropy: 0.3393
2025-06-07 00:34:09 [INFO]: Epoch 008 - training loss (CrossEntropy): 1573.3481, validation CrossEntropy: 0.3782
2025-06-07 00:34:13 [INFO]: Epoch 009 - training loss (CrossEntropy): 1565.6619, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=1) & 0.6057 ± 0.0945 & 0.2822 ± 0.3077 0.4591 ± 0.3766 0.2379 ± 0.2761 \
[143.43686366081238, 100.86256146430969, 142.1179211139679, 143.1390471458435, 143.97495126724243]
134.7063 ± 16.9326


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=16
classification_weight=1
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 00:35:54 [INFO]: No given device, using default device: cpu
2025-06-07 00:35:54 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T003554
2025-06-07 00:35:54 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T003554/tensorboard
2025-06-07 00:35:54 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 00:35:54 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 00:35:54 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 8,564
2025-06-07 00:36:02 [INFO]: Epoch 001 - training loss (CrossEntropy): 2526.3798, validation CrossEntropy: 0.6625
2025-06-07 00:36:07 [INFO]: Epoch 002 - training loss (CrossEntropy): 1989.2775, validation CrossEntropy: 0.5747
2025-06-07 00:36:12 [INFO]: Epoch 003 - training loss (CrossEntropy): 1736.2647, validation CrossEntropy: 0.5432
2025-06-07 00:36:16 [INFO]: Epoch 004 - training loss

BRITS(rnn_hidden_size=16, classification_weight=1, reconstruction_weight=1) & 0.5241 ± 0.0000 & 0.0000 ± 0.0000 0.0000 ± 0.0000 0.0000 ± 0.0000 \
[138.47289037704468]
138.4729 ± 0.0000


2025-06-07 00:38:24 [INFO]: Epoch 001 - training loss (CrossEntropy): 2372.1790, validation CrossEntropy: 0.6741
2025-06-07 00:38:29 [INFO]: Epoch 002 - training loss (CrossEntropy): 1935.4435, validation CrossEntropy: 0.6101
2025-06-07 00:38:33 [INFO]: Epoch 003 - training loss (CrossEntropy): 1730.1801, validation CrossEntropy: 0.5385
2025-06-07 00:38:37 [INFO]: Epoch 004 - training loss (CrossEntropy): 1625.3295, validation CrossEntropy: 0.5327
2025-06-07 00:38:42 [INFO]: Epoch 005 - training loss (CrossEntropy): 1592.5259, validation CrossEntropy: 0.4646
2025-06-07 00:38:46 [INFO]: Epoch 006 - training loss (CrossEntropy): 1589.6318, validation CrossEntropy: 0.4574
2025-06-07 00:38:50 [INFO]: Epoch 007 - training loss (CrossEntropy): 1583.7845, validation CrossEntropy: 0.4570
2025-06-07 00:38:56 [INFO]: Epoch 008 - training loss (CrossEntropy): 1576.5617, validation CrossEntropy: 0.4963
2025-06-07 00:39:00 [INFO]: Epoch 009 - training loss (CrossEntropy): 1574.4309, validation Cros

BRITS(rnn_hidden_size=16, classification_weight=1, reconstruction_weight=1) & 0.5731 ± 0.0490 & 0.1986 ± 0.1986 0.4123 ± 0.4123 0.1308 ± 0.1308 \
[138.47289037704468, 138.54999613761902]
138.5114 ± 0.0386


2025-06-07 00:40:46 [INFO]: Epoch 001 - training loss (CrossEntropy): 2446.4357, validation CrossEntropy: 0.6802
2025-06-07 00:40:50 [INFO]: Epoch 002 - training loss (CrossEntropy): 1975.3862, validation CrossEntropy: 0.6041
2025-06-07 00:40:54 [INFO]: Epoch 003 - training loss (CrossEntropy): 1756.8523, validation CrossEntropy: 0.5238
2025-06-07 00:40:59 [INFO]: Epoch 004 - training loss (CrossEntropy): 1650.7338, validation CrossEntropy: 0.4980
2025-06-07 00:41:03 [INFO]: Epoch 005 - training loss (CrossEntropy): 1595.4602, validation CrossEntropy: 0.4784
2025-06-07 00:41:08 [INFO]: Epoch 006 - training loss (CrossEntropy): 1586.5067, validation CrossEntropy: 0.4181
2025-06-07 00:41:13 [INFO]: Epoch 007 - training loss (CrossEntropy): 1578.4526, validation CrossEntropy: 0.4268
2025-06-07 00:41:17 [INFO]: Epoch 008 - training loss (CrossEntropy): 1577.5084, validation CrossEntropy: 0.4362
2025-06-07 00:41:21 [INFO]: Epoch 009 - training loss (CrossEntropy): 1570.8545, validation Cros

BRITS(rnn_hidden_size=16, classification_weight=1, reconstruction_weight=1) & 0.5762 ± 0.0402 & 0.2177 ± 0.1644 0.5550 ± 0.3925 0.1375 ± 0.1072 \
[138.47289037704468, 138.54999613761902, 138.8585832118988]
138.6272 ± 0.1666


2025-06-07 00:43:08 [INFO]: Epoch 001 - training loss (CrossEntropy): 2628.5608, validation CrossEntropy: 0.6067
2025-06-07 00:43:12 [INFO]: Epoch 002 - training loss (CrossEntropy): 2111.2661, validation CrossEntropy: 0.5412
2025-06-07 00:43:17 [INFO]: Epoch 003 - training loss (CrossEntropy): 1826.5777, validation CrossEntropy: 0.4726
2025-06-07 00:43:21 [INFO]: Epoch 004 - training loss (CrossEntropy): 1662.8903, validation CrossEntropy: 0.4128
2025-06-07 00:43:27 [INFO]: Epoch 005 - training loss (CrossEntropy): 1595.1700, validation CrossEntropy: 0.3923
2025-06-07 00:43:32 [INFO]: Epoch 006 - training loss (CrossEntropy): 1587.9177, validation CrossEntropy: 0.3809
2025-06-07 00:43:36 [INFO]: Epoch 007 - training loss (CrossEntropy): 1581.3388, validation CrossEntropy: 0.3969
2025-06-07 00:43:40 [INFO]: Epoch 008 - training loss (CrossEntropy): 1576.9907, validation CrossEntropy: 0.3568
2025-06-07 00:43:45 [INFO]: Epoch 009 - training loss (CrossEntropy): 1572.2459, validation Cros

BRITS(rnn_hidden_size=16, classification_weight=1, reconstruction_weight=1) & 0.5632 ± 0.0415 & 0.1632 ± 0.1707 0.4163 ± 0.4163 0.1031 ± 0.1103 \
[138.47289037704468, 138.54999613761902, 138.8585832118988, 139.1692705154419]
138.7627 ± 0.2756


2025-06-07 00:45:31 [INFO]: Epoch 001 - training loss (CrossEntropy): 2691.6039, validation CrossEntropy: 0.6377
2025-06-07 00:45:37 [INFO]: Epoch 002 - training loss (CrossEntropy): 2146.9382, validation CrossEntropy: 0.5617
2025-06-07 00:45:41 [INFO]: Epoch 003 - training loss (CrossEntropy): 1834.9724, validation CrossEntropy: 0.5316
2025-06-07 00:45:45 [INFO]: Epoch 004 - training loss (CrossEntropy): 1657.8820, validation CrossEntropy: 0.4342
2025-06-07 00:45:50 [INFO]: Epoch 005 - training loss (CrossEntropy): 1596.1472, validation CrossEntropy: 0.3967
2025-06-07 00:45:54 [INFO]: Epoch 006 - training loss (CrossEntropy): 1585.8282, validation CrossEntropy: 0.4078
2025-06-07 00:45:58 [INFO]: Epoch 007 - training loss (CrossEntropy): 1579.8803, validation CrossEntropy: 0.4158
2025-06-07 00:46:03 [INFO]: Epoch 008 - training loss (CrossEntropy): 1577.2056, validation CrossEntropy: 0.4195
2025-06-07 00:46:08 [INFO]: Epoch 009 - training loss (CrossEntropy): 1572.9819, validation Cros

BRITS(rnn_hidden_size=16, classification_weight=1, reconstruction_weight=1) & 0.5820 ± 0.0529 & 0.2222 ± 0.1930 0.5180 ± 0.4243 0.1434 ± 0.1274 \
[138.47289037704468, 138.54999613761902, 138.8585832118988, 139.1692705154419, 138.9210798740387]
138.7944 ± 0.2545


In [ ]:

rnn_hidden_size=64
classification_weight=2
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 00:47:45 [INFO]: No given device, using default device: cpu
2025-06-07 00:47:45 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T004745
2025-06-07 00:47:45 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T004745/tensorboard
2025-06-07 00:47:45 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 00:47:45 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 00:47:45 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 53,780
2025-06-07 00:47:55 [INFO]: Epoch 001 - training loss (CrossEntropy): 2595.4029, validation CrossEntropy: 0.3768
2025-06-07 00:48:00 [INFO]: Epoch 002 - training loss (CrossEntropy): 2092.2342, validation CrossEntropy: 0.3298
2025-06-07 00:48:05 [INFO]: Epoch 003 - training loss (CrossEntropy): 1815.3398, validation CrossEntropy: 0.3189
2025-06-07 00:48:10 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=64, classification_weight=2, reconstruction_weight=1) & 0.5246 ± 0.0000 & 0.0055 ± 0.0000 0.6250 ± 0.0000 0.0028 ± 0.0000 \
[160.4247431755066]
160.4247 ± 0.0000


2025-06-07 00:50:38 [INFO]: Epoch 001 - training loss (CrossEntropy): 2571.8620, validation CrossEntropy: 0.4342
2025-06-07 00:50:43 [INFO]: Epoch 002 - training loss (CrossEntropy): 2090.8780, validation CrossEntropy: 0.3242
2025-06-07 00:50:48 [INFO]: Epoch 003 - training loss (CrossEntropy): 1816.0051, validation CrossEntropy: 0.3286
2025-06-07 00:50:54 [INFO]: Epoch 004 - training loss (CrossEntropy): 1652.3698, validation CrossEntropy: 0.2803
2025-06-07 00:50:58 [INFO]: Epoch 005 - training loss (CrossEntropy): 1589.0813, validation CrossEntropy: 0.2799
2025-06-07 00:51:04 [INFO]: Epoch 006 - training loss (CrossEntropy): 1579.1826, validation CrossEntropy: 0.2493
2025-06-07 00:51:09 [INFO]: Epoch 007 - training loss (CrossEntropy): 1573.1469, validation CrossEntropy: 0.2343
2025-06-07 00:51:14 [INFO]: Epoch 008 - training loss (CrossEntropy): 1569.9280, validation CrossEntropy: 0.2116
2025-06-07 00:51:21 [INFO]: Epoch 009 - training loss (CrossEntropy): 1566.1581, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=2, reconstruction_weight=1) & 0.6253 ± 0.1006 & 0.3131 ± 0.3076 0.7669 ± 0.1419 0.2371 ± 0.2343 \
[160.4247431755066, 158.54039239883423]
159.4826 ± 0.9422


2025-06-07 00:53:21 [INFO]: Epoch 001 - training loss (CrossEntropy): 2755.0618, validation CrossEntropy: 0.4241
2025-06-07 00:53:27 [INFO]: Epoch 002 - training loss (CrossEntropy): 2182.5111, validation CrossEntropy: 0.2881
2025-06-07 00:53:32 [INFO]: Epoch 003 - training loss (CrossEntropy): 1858.8633, validation CrossEntropy: 0.2930
2025-06-07 00:53:38 [INFO]: Epoch 004 - training loss (CrossEntropy): 1665.7722, validation CrossEntropy: 0.2719
2025-06-07 00:53:42 [INFO]: Epoch 005 - training loss (CrossEntropy): 1596.0449, validation CrossEntropy: 0.2436
2025-06-07 00:53:47 [INFO]: Epoch 006 - training loss (CrossEntropy): 1584.3968, validation CrossEntropy: 0.2282
2025-06-07 00:53:53 [INFO]: Epoch 007 - training loss (CrossEntropy): 1580.5326, validation CrossEntropy: 0.2371
2025-06-07 00:53:58 [INFO]: Epoch 008 - training loss (CrossEntropy): 1575.8341, validation CrossEntropy: 0.2163
2025-06-07 00:54:03 [INFO]: Epoch 009 - training loss (CrossEntropy): 1569.3597, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=2, reconstruction_weight=1) & 0.5915 ± 0.0950 & 0.2087 ± 0.2913 0.5112 ± 0.3796 0.1580 ± 0.2216 \
[160.4247431755066, 158.54039239883423, 124.86591601371765]
147.9437 ± 16.3366


2025-06-07 00:55:30 [INFO]: Epoch 001 - training loss (CrossEntropy): 2684.9186, validation CrossEntropy: 0.4143
2025-06-07 00:55:35 [INFO]: Epoch 002 - training loss (CrossEntropy): 2139.8100, validation CrossEntropy: 0.3384
2025-06-07 00:55:40 [INFO]: Epoch 003 - training loss (CrossEntropy): 1821.4496, validation CrossEntropy: 0.2976
2025-06-07 00:55:45 [INFO]: Epoch 004 - training loss (CrossEntropy): 1647.7128, validation CrossEntropy: 0.2812
2025-06-07 00:55:50 [INFO]: Epoch 005 - training loss (CrossEntropy): 1594.0171, validation CrossEntropy: 0.2843
2025-06-07 00:55:55 [INFO]: Epoch 006 - training loss (CrossEntropy): 1579.6393, validation CrossEntropy: 0.2401
2025-06-07 00:56:00 [INFO]: Epoch 007 - training loss (CrossEntropy): 1576.6246, validation CrossEntropy: 0.2305
2025-06-07 00:56:04 [INFO]: Epoch 008 - training loss (CrossEntropy): 1572.4172, validation CrossEntropy: 0.2170
2025-06-07 00:56:10 [INFO]: Epoch 009 - training loss (CrossEntropy): 1563.8273, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=2, reconstruction_weight=1) & 0.5863 ± 0.0828 & 0.2032 ± 0.2525 0.6208 ± 0.3796 0.1444 ± 0.1933 \
[160.4247431755066, 158.54039239883423, 124.86591601371765, 157.4579632282257]
150.3223 ± 14.7355


2025-06-07 00:58:09 [INFO]: Epoch 001 - training loss (CrossEntropy): 2376.8163, validation CrossEntropy: 0.4322
2025-06-07 00:58:15 [INFO]: Epoch 002 - training loss (CrossEntropy): 1981.4563, validation CrossEntropy: 0.3311
2025-06-07 00:58:20 [INFO]: Epoch 003 - training loss (CrossEntropy): 1772.9728, validation CrossEntropy: 0.3073
2025-06-07 00:58:26 [INFO]: Epoch 004 - training loss (CrossEntropy): 1649.5965, validation CrossEntropy: 0.2766
2025-06-07 00:58:31 [INFO]: Epoch 005 - training loss (CrossEntropy): 1593.4791, validation CrossEntropy: 0.2639
2025-06-07 00:58:35 [INFO]: Epoch 006 - training loss (CrossEntropy): 1578.2004, validation CrossEntropy: 0.2437
2025-06-07 00:58:41 [INFO]: Epoch 007 - training loss (CrossEntropy): 1574.5474, validation CrossEntropy: 0.2277
2025-06-07 00:58:46 [INFO]: Epoch 008 - training loss (CrossEntropy): 1570.7954, validation CrossEntropy: 0.2097
2025-06-07 00:58:51 [INFO]: Epoch 009 - training loss (CrossEntropy): 1564.0701, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=2, reconstruction_weight=1) & 0.5814 ± 0.0747 & 0.1923 ± 0.2269 0.6939 ± 0.3697 0.1316 ± 0.1748 \
[160.4247431755066, 158.54039239883423, 124.86591601371765, 157.4579632282257, 158.18945503234863]
151.8957 ± 13.5503


In [ ]:

rnn_hidden_size=32
classification_weight=2
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 01:00:43 [INFO]: No given device, using default device: cpu
2025-06-07 01:00:43 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T010043
2025-06-07 01:00:43 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T010043/tensorboard
2025-06-07 01:00:43 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 01:00:43 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 01:00:43 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,540
2025-06-07 01:00:51 [INFO]: Epoch 001 - training loss (CrossEntropy): 2694.5495, validation CrossEntropy: 0.5144
2025-06-07 01:00:56 [INFO]: Epoch 002 - training loss (CrossEntropy): 2127.5777, validation CrossEntropy: 0.3789
2025-06-07 01:01:02 [INFO]: Epoch 003 - training loss (CrossEntropy): 1845.2241, validation CrossEntropy: 0.3203
2025-06-07 01:01:07 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=32, classification_weight=2, reconstruction_weight=1) & 0.5288 ± 0.0000 & 0.0196 ± 0.0000 1.0000 ± 0.0000 0.0099 ± 0.0000 \
[112.16539740562439]
112.1654 ± 0.0000


2025-06-07 01:02:46 [INFO]: Epoch 001 - training loss (CrossEntropy): 2539.0163, validation CrossEntropy: 0.5107
2025-06-07 01:02:51 [INFO]: Epoch 002 - training loss (CrossEntropy): 1985.3575, validation CrossEntropy: 0.3982
2025-06-07 01:02:57 [INFO]: Epoch 003 - training loss (CrossEntropy): 1743.4451, validation CrossEntropy: 0.3940
2025-06-07 01:03:02 [INFO]: Epoch 004 - training loss (CrossEntropy): 1640.0433, validation CrossEntropy: 0.3322
2025-06-07 01:03:07 [INFO]: Epoch 005 - training loss (CrossEntropy): 1593.7294, validation CrossEntropy: 0.3460
2025-06-07 01:03:11 [INFO]: Epoch 006 - training loss (CrossEntropy): 1583.2397, validation CrossEntropy: 0.3271
2025-06-07 01:03:16 [INFO]: Epoch 007 - training loss (CrossEntropy): 1580.7788, validation CrossEntropy: 0.2988
2025-06-07 01:03:20 [INFO]: Epoch 008 - training loss (CrossEntropy): 1577.2108, validation CrossEntropy: 0.2808
2025-06-07 01:03:25 [INFO]: Epoch 009 - training loss (CrossEntropy): 1571.0597, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=2, reconstruction_weight=1) & 0.5265 ± 0.0024 & 0.0098 ± 0.0098 0.5000 ± 0.5000 0.0050 ± 0.0050 \
[112.16539740562439, 145.59908175468445]
128.8822 ± 16.7168


2025-06-07 01:05:16 [INFO]: Epoch 001 - training loss (CrossEntropy): 2588.3286, validation CrossEntropy: 0.5587
2025-06-07 01:05:21 [INFO]: Epoch 002 - training loss (CrossEntropy): 2078.2976, validation CrossEntropy: 0.4126
2025-06-07 01:05:26 [INFO]: Epoch 003 - training loss (CrossEntropy): 1811.1414, validation CrossEntropy: 0.3669
2025-06-07 01:05:30 [INFO]: Epoch 004 - training loss (CrossEntropy): 1655.4283, validation CrossEntropy: 0.3689
2025-06-07 01:05:35 [INFO]: Epoch 005 - training loss (CrossEntropy): 1593.6309, validation CrossEntropy: 0.3291
2025-06-07 01:05:39 [INFO]: Epoch 006 - training loss (CrossEntropy): 1586.3501, validation CrossEntropy: 0.2973
2025-06-07 01:05:44 [INFO]: Epoch 007 - training loss (CrossEntropy): 1580.0288, validation CrossEntropy: 0.3106
2025-06-07 01:05:49 [INFO]: Epoch 008 - training loss (CrossEntropy): 1574.0567, validation CrossEntropy: 0.2825
2025-06-07 01:05:53 [INFO]: Epoch 009 - training loss (CrossEntropy): 1571.0089, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=2, reconstruction_weight=1) & 0.5687 ± 0.0598 & 0.1697 ± 0.2262 0.6060 ± 0.4349 0.1197 ± 0.1623 \
[112.16539740562439, 145.59908175468445, 143.3373670578003]
133.7006 ± 15.2557


2025-06-07 01:07:43 [INFO]: Epoch 001 - training loss (CrossEntropy): 2563.0212, validation CrossEntropy: 0.5834
2025-06-07 01:07:47 [INFO]: Epoch 002 - training loss (CrossEntropy): 2033.6742, validation CrossEntropy: 0.4193
2025-06-07 01:07:53 [INFO]: Epoch 003 - training loss (CrossEntropy): 1793.4865, validation CrossEntropy: 0.3507
2025-06-07 01:07:57 [INFO]: Epoch 004 - training loss (CrossEntropy): 1661.0668, validation CrossEntropy: 0.3467
2025-06-07 01:08:01 [INFO]: Epoch 005 - training loss (CrossEntropy): 1597.2204, validation CrossEntropy: 0.3095
2025-06-07 01:08:07 [INFO]: Epoch 006 - training loss (CrossEntropy): 1582.0965, validation CrossEntropy: 0.2910
2025-06-07 01:08:11 [INFO]: Epoch 007 - training loss (CrossEntropy): 1578.8673, validation CrossEntropy: 0.2916
2025-06-07 01:08:15 [INFO]: Epoch 008 - training loss (CrossEntropy): 1577.8696, validation CrossEntropy: 0.3074
2025-06-07 01:08:21 [INFO]: Epoch 009 - training loss (CrossEntropy): 1569.3944, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=2, reconstruction_weight=1) & 0.5576 ± 0.0553 & 0.1273 ± 0.2092 0.4545 ± 0.4590 0.0898 ± 0.1498 \
[112.16539740562439, 145.59908175468445, 143.3373670578003, 146.5832281112671]
136.9213 ± 14.3412


2025-06-07 01:10:12 [INFO]: Epoch 001 - training loss (CrossEntropy): 2553.4146, validation CrossEntropy: 0.4902
2025-06-07 01:10:17 [INFO]: Epoch 002 - training loss (CrossEntropy): 2044.0326, validation CrossEntropy: 0.3702
2025-06-07 01:10:21 [INFO]: Epoch 003 - training loss (CrossEntropy): 1768.4367, validation CrossEntropy: 0.3117
2025-06-07 01:10:26 [INFO]: Epoch 004 - training loss (CrossEntropy): 1618.2230, validation CrossEntropy: 0.3168
2025-06-07 01:10:31 [INFO]: Epoch 005 - training loss (CrossEntropy): 1590.5633, validation CrossEntropy: 0.3175
2025-06-07 01:10:35 [INFO]: Epoch 006 - training loss (CrossEntropy): 1582.5889, validation CrossEntropy: 0.3025
2025-06-07 01:10:41 [INFO]: Epoch 007 - training loss (CrossEntropy): 1580.4059, validation CrossEntropy: 0.2970
2025-06-07 01:10:46 [INFO]: Epoch 008 - training loss (CrossEntropy): 1574.4592, validation CrossEntropy: 0.2776
2025-06-07 01:10:51 [INFO]: Epoch 009 - training loss (CrossEntropy): 1571.5772, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=2, reconstruction_weight=1) & 0.5514 ± 0.0510 & 0.1053 ± 0.1922 0.5091 ± 0.4248 0.0736 ± 0.1378 \
[112.16539740562439, 145.59908175468445, 143.3373670578003, 146.5832281112671, 146.0697319507599]
138.7510 ± 13.3389


In [ ]:

rnn_hidden_size=64
classification_weight=1
reconstruction_weight=2

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 01:12:32 [INFO]: No given device, using default device: cpu
2025-06-07 01:12:32 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T011232
2025-06-07 01:12:32 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T011232/tensorboard
2025-06-07 01:12:32 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 01:12:32 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 01:12:32 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 53,780
2025-06-07 01:12:43 [INFO]: Epoch 001 - training loss (CrossEntropy): 5059.3433, validation CrossEntropy: 0.4792
2025-06-07 01:12:48 [INFO]: Epoch 002 - training loss (CrossEntropy): 4085.2162, validation CrossEntropy: 0.4504
2025-06-07 01:12:53 [INFO]: Epoch 003 - training loss (CrossEntropy): 3623.1096, validation CrossEntropy: 0.4479
2025-06-07 01:12:59 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=2) & 0.5241 ± 0.0000 & 0.0000 ± 0.0000 0.0000 ± 0.0000 0.0000 ± 0.0000 \
[161.34328961372375]
161.3433 ± 0.0000


2025-06-07 01:15:26 [INFO]: Epoch 001 - training loss (CrossEntropy): 4962.7280, validation CrossEntropy: 0.4799
2025-06-07 01:15:31 [INFO]: Epoch 002 - training loss (CrossEntropy): 4048.1058, validation CrossEntropy: 0.4723
2025-06-07 01:15:36 [INFO]: Epoch 003 - training loss (CrossEntropy): 3553.2744, validation CrossEntropy: 0.4427
2025-06-07 01:15:42 [INFO]: Epoch 004 - training loss (CrossEntropy): 3263.5407, validation CrossEntropy: 0.4410
2025-06-07 01:15:46 [INFO]: Epoch 005 - training loss (CrossEntropy): 3179.0986, validation CrossEntropy: 0.3769
2025-06-07 01:15:51 [INFO]: Epoch 006 - training loss (CrossEntropy): 3166.0997, validation CrossEntropy: 0.3609
2025-06-07 01:15:57 [INFO]: Epoch 007 - training loss (CrossEntropy): 3156.6502, validation CrossEntropy: 0.3182
2025-06-07 01:16:01 [INFO]: Epoch 008 - training loss (CrossEntropy): 3140.6811, validation CrossEntropy: 0.3247
2025-06-07 01:16:07 [INFO]: Epoch 009 - training loss (CrossEntropy): 3134.0244, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=2) & 0.6395 ± 0.1154 & 0.3446 ± 0.3446 0.4346 ± 0.4346 0.2855 ± 0.2855 \
[161.34328961372375, 156.93805027008057]
159.1407 ± 2.2026


2025-06-07 01:18:07 [INFO]: Epoch 001 - training loss (CrossEntropy): 5514.1618, validation CrossEntropy: 0.4378
2025-06-07 01:18:13 [INFO]: Epoch 002 - training loss (CrossEntropy): 4380.3843, validation CrossEntropy: 0.4127
2025-06-07 01:18:17 [INFO]: Epoch 003 - training loss (CrossEntropy): 3773.1580, validation CrossEntropy: 0.4142
2025-06-07 01:18:22 [INFO]: Epoch 004 - training loss (CrossEntropy): 3421.5843, validation CrossEntropy: 0.4340
2025-06-07 01:18:29 [INFO]: Epoch 005 - training loss (CrossEntropy): 3208.2886, validation CrossEntropy: 0.4059
2025-06-07 01:18:34 [INFO]: Epoch 006 - training loss (CrossEntropy): 3164.6601, validation CrossEntropy: 0.3822
2025-06-07 01:18:39 [INFO]: Epoch 007 - training loss (CrossEntropy): 3154.0193, validation CrossEntropy: 0.3587
2025-06-07 01:18:44 [INFO]: Epoch 008 - training loss (CrossEntropy): 3154.2328, validation CrossEntropy: 0.3940
2025-06-07 01:18:49 [INFO]: Epoch 009 - training loss (CrossEntropy): 3140.9971, validation Cros

...

2025-06-07 01:20:45 [INFO]: Epoch 001 - training loss (CrossEntropy): 5410.3208, validation CrossEntropy: 0.4658
2025-06-07 01:20:50 [INFO]: Epoch 002 - training loss (CrossEntropy): 4322.4871, validation CrossEntropy: 0.4085
2025-06-07 01:20:55 [INFO]: Epoch 003 - training loss (CrossEntropy): 3725.9347, validation CrossEntropy: 0.3914
2025-06-07 01:21:00 [INFO]: Epoch 004 - training loss (CrossEntropy): 3354.5715, validation CrossEntropy: 0.4074
2025-06-07 01:21:05 [INFO]: Epoch 005 - training loss (CrossEntropy): 3194.3464, validation CrossEntropy: 0.3993
2025-06-07 01:21:10 [INFO]: Epoch 006 - training loss (CrossEntropy): 3170.8232, validation CrossEntropy: 0.3707
2025-06-07 01:21:15 [INFO]: Epoch 007 - training loss (CrossEntropy): 3162.2586, validation CrossEntropy: 0.3544
2025-06-07 01:21:20 [INFO]: Epoch 008 - training loss (CrossEntropy): 3153.4026, validation CrossEntropy: 0.3295
2025-06-07 01:21:26 [INFO]: Epoch 009 - training loss (CrossEntropy): 3138.5764, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=2) & 0.6327 ± 0.1090 & 0.3271 ± 0.3280 0.4489 ± 0.4494 0.2589 ± 0.2617 \
[161.34328961372375, 156.93805027008057, 154.2410068511963, 157.30895733833313]
157.4578 ± 2.5366


2025-06-07 01:23:25 [INFO]: Epoch 001 - training loss (CrossEntropy): 5188.6403, validation CrossEntropy: 0.5052
2025-06-07 01:23:31 [INFO]: Epoch 002 - training loss (CrossEntropy): 4146.4963, validation CrossEntropy: 0.5411
2025-06-07 01:23:36 [INFO]: Epoch 003 - training loss (CrossEntropy): 3621.4256, validation CrossEntropy: 0.5057
2025-06-07 01:23:40 [INFO]: Epoch 004 - training loss (CrossEntropy): 3304.9009, validation CrossEntropy: 0.5131
2025-06-07 01:23:46 [INFO]: Epoch 005 - training loss (CrossEntropy): 3186.4768, validation CrossEntropy: 0.4460
2025-06-07 01:23:51 [INFO]: Epoch 006 - training loss (CrossEntropy): 3169.1770, validation CrossEntropy: 0.4140
2025-06-07 01:23:56 [INFO]: Epoch 007 - training loss (CrossEntropy): 3163.9522, validation CrossEntropy: 0.3678
2025-06-07 01:24:01 [INFO]: Epoch 008 - training loss (CrossEntropy): 3153.0408, validation CrossEntropy: 0.3521
2025-06-07 01:24:06 [INFO]: Epoch 009 - training loss (CrossEntropy): 3140.9663, validation Cros

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=2) & 0.6110 ± 0.1068 & 0.2617 ± 0.3212 0.3591 ± 0.4402 0.2072 ± 0.2559 \
[161.34328961372375, 156.93805027008057, 154.2410068511963, 157.30895733833313, 157.66619682312012]
157.4995 ± 2.2703


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=32
classification_weight=1
reconstruction_weight=2

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 01:25:58 [INFO]: No given device, using default device: cpu
2025-06-07 01:25:58 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T012558
2025-06-07 01:25:58 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T012558/tensorboard
2025-06-07 01:25:58 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 01:25:58 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 01:25:58 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,540
2025-06-07 01:26:07 [INFO]: Epoch 001 - training loss (CrossEntropy): 4712.4060, validation CrossEntropy: 0.5548
2025-06-07 01:26:13 [INFO]: Epoch 002 - training loss (CrossEntropy): 3866.4157, validation CrossEntropy: 0.5794
2025-06-07 01:26:19 [INFO]: Epoch 003 - training loss (CrossEntropy): 3470.8662, validation CrossEntropy: 0.6179
2025-06-07 01:26:23 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=2) & 0.8092 ± 0.0000 & 0.8098 ± 0.0000 0.7704 ± 0.0000 0.8535 ± 0.0000 \
[147.9333107471466]
147.9333 ± 0.0000


2025-06-07 01:28:38 [INFO]: Epoch 001 - training loss (CrossEntropy): 4827.6173, validation CrossEntropy: 0.5957
2025-06-07 01:28:42 [INFO]: Epoch 002 - training loss (CrossEntropy): 3883.1556, validation CrossEntropy: 0.5410
2025-06-07 01:28:48 [INFO]: Epoch 003 - training loss (CrossEntropy): 3479.0708, validation CrossEntropy: 0.5200
2025-06-07 01:28:52 [INFO]: Epoch 004 - training loss (CrossEntropy): 3258.7033, validation CrossEntropy: 0.4801
2025-06-07 01:28:56 [INFO]: Epoch 005 - training loss (CrossEntropy): 3186.7105, validation CrossEntropy: 0.5299
2025-06-07 01:29:02 [INFO]: Epoch 006 - training loss (CrossEntropy): 3162.8955, validation CrossEntropy: 0.5062
2025-06-07 01:29:06 [INFO]: Epoch 007 - training loss (CrossEntropy): 3156.2997, validation CrossEntropy: 0.4830
2025-06-07 01:29:10 [INFO]: Epoch 008 - training loss (CrossEntropy): 3146.6331, validation CrossEntropy: 0.4797
2025-06-07 01:29:16 [INFO]: Epoch 009 - training loss (CrossEntropy): 3134.8550, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=2) & 0.6673 ± 0.1419 & 0.4077 ± 0.4022 0.8852 ± 0.1148 0.4281 ± 0.4254 \
[147.9333107471466, 144.7734797000885]
146.3534 ± 1.5799


2025-06-07 01:31:05 [INFO]: Epoch 001 - training loss (CrossEntropy): 5404.9144, validation CrossEntropy: 0.5433
2025-06-07 01:31:09 [INFO]: Epoch 002 - training loss (CrossEntropy): 4313.6219, validation CrossEntropy: 0.5018
2025-06-07 01:31:14 [INFO]: Epoch 003 - training loss (CrossEntropy): 3753.1476, validation CrossEntropy: 0.4612
2025-06-07 01:31:19 [INFO]: Epoch 004 - training loss (CrossEntropy): 3398.6379, validation CrossEntropy: 0.5473
2025-06-07 01:31:23 [INFO]: Epoch 005 - training loss (CrossEntropy): 3201.4386, validation CrossEntropy: 0.5248
2025-06-07 01:31:28 [INFO]: Epoch 006 - training loss (CrossEntropy): 3177.6932, validation CrossEntropy: 0.5240
2025-06-07 01:31:33 [INFO]: Epoch 007 - training loss (CrossEntropy): 3162.1305, validation CrossEntropy: 0.4880
2025-06-07 01:31:37 [INFO]: Epoch 008 - training loss (CrossEntropy): 3151.9911, validation CrossEntropy: 0.4882
2025-06-07 01:31:42 [INFO]: Epoch 009 - training loss (CrossEntropy): 3144.1931, validation Cros

...

2025-06-07 01:31:53 [INFO]: Epoch 001 - training loss (CrossEntropy): 5409.4458, validation CrossEntropy: 0.5392
2025-06-07 01:32:00 [INFO]: Epoch 002 - training loss (CrossEntropy): 4341.4749, validation CrossEntropy: 0.4830
2025-06-07 01:32:05 [INFO]: Epoch 003 - training loss (CrossEntropy): 3767.1340, validation CrossEntropy: 0.5254
2025-06-07 01:32:09 [INFO]: Epoch 004 - training loss (CrossEntropy): 3394.6491, validation CrossEntropy: 0.5283
2025-06-07 01:32:14 [INFO]: Epoch 005 - training loss (CrossEntropy): 3203.8597, validation CrossEntropy: 0.4823
2025-06-07 01:32:18 [INFO]: Epoch 006 - training loss (CrossEntropy): 3174.6830, validation CrossEntropy: 0.4291
2025-06-07 01:32:23 [INFO]: Epoch 007 - training loss (CrossEntropy): 3165.7636, validation CrossEntropy: 0.3994
2025-06-07 01:32:28 [INFO]: Epoch 008 - training loss (CrossEntropy): 3152.7176, validation CrossEntropy: 0.4654
2025-06-07 01:32:32 [INFO]: Epoch 009 - training loss (CrossEntropy): 3147.9258, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=2) & 0.5957 ± 0.1233 & 0.2038 ± 0.3499 0.4426 ± 0.4500 0.2141 ± 0.3692 \
[147.9333107471466, 144.7734797000885, 45.52221369743347, 64.25737762451172]
100.6216 ± 46.2225


2025-06-07 01:33:02 [INFO]: Epoch 001 - training loss (CrossEntropy): 5286.6580, validation CrossEntropy: 0.5983
2025-06-07 01:33:07 [INFO]: Epoch 002 - training loss (CrossEntropy): 4208.4129, validation CrossEntropy: 0.4965
2025-06-07 01:33:12 [INFO]: Epoch 003 - training loss (CrossEntropy): 3613.1855, validation CrossEntropy: 0.5000
2025-06-07 01:33:16 [INFO]: Epoch 004 - training loss (CrossEntropy): 3307.4766, validation CrossEntropy: 0.5613
2025-06-07 01:33:21 [INFO]: Epoch 005 - training loss (CrossEntropy): 3192.6851, validation CrossEntropy: 0.5349
2025-06-07 01:33:25 [INFO]: Epoch 006 - training loss (CrossEntropy): 3173.8327, validation CrossEntropy: 0.5159
2025-06-07 01:33:29 [INFO]: Epoch 007 - training loss (CrossEntropy): 3158.0301, validation CrossEntropy: 0.4957
2025-06-07 01:33:35 [INFO]: Epoch 008 - training loss (CrossEntropy): 3147.1031, validation CrossEntropy: 0.4526
2025-06-07 01:33:39 [INFO]: Epoch 009 - training loss (CrossEntropy): 3135.2636, validation Cros

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=2) & 0.5814 ± 0.1139 & 0.1631 ± 0.3234 0.3541 ± 0.4397 0.1713 ± 0.3411 \
[147.9333107471466, 144.7734797000885, 45.52221369743347, 64.25737762451172, 143.15850162506104]
109.1290 ± 44.7070


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Not deep models

In [38]:
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.base import ClassifierMixin
from typing import Any, TypeVar
from collections import namedtuple
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from time import time

import ray
from ray import tune


In [29]:
_T = TypeVar('_T', bound=ClassifierMixin)

def evaluate_classic_model(model: _T, X: Any, y: Any):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='macro')
    precision = precision_score(y, y_pred, average='macro')
    recall = recall_score(y, y_pred, average='macro')
    confusion = confusion_matrix(y, y_pred)
    return namedtuple('Evaluation', ['accuracy', 'f1', 'precision', 'recall', 'confusion'])(accuracy, f1, precision, recall, confusion)

def _get_formatted_metric(values: list[float]):
  avg = sum(values) / len(values)
  std = (sum([(v - avg) ** 2 for v in values]) / len(values)) ** 0.5
  return f"{avg:.4f} ± {std:.4f}"


def run_classic_model(
        model_cls: _T,
        model_kwargs: dict[str, Any],
        X_train, y_train, X_val, y_val, X_test, y_test,
        repeat: int = 5
):
  if X_train.ndim == 3:
      X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
  if X_val.ndim == 3:
      X_val = X_val.reshape((X_val.shape[0], X_val.shape[1] * X_val.shape[2]))
  if X_test.ndim == 3:
      X_test = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

  final_res = {
      'accuracy': [],
      'f1': [],
      'precision': [],
      'recall': []
  }
  training_times = []
  for _ in range(repeat):
    model = model_cls(**model_kwargs)
    _start = time()
    model.fit(X_train, y_train)
    _end = time()
    training_times.append(_end - _start)
    results = evaluate_classic_model(model, X_test, y_test)
    final_res['accuracy'].append(results.accuracy)
    final_res['f1'].append(results.f1)
    final_res['precision'].append(results.precision)
    final_res['recall'].append(results.recall)

  print(
    f"{model_cls} & {_get_formatted_metric(final_res['accuracy'])} & {_get_formatted_metric(final_res['f1'])} {_get_formatted_metric(final_res['precision'])} {_get_formatted_metric(final_res['recall'])} \\"
  )
  print(training_times)
  print(_get_formatted_metric(training_times))


In [47]:
def train_model_tune(
    config,
    model_cls: _T,
    X_train, y_train,
    X_val, y_val
):
  if X_train.ndim == 3:
      X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
  if X_val.ndim == 3:
      X_val = X_val.reshape((X_val.shape[0], X_val.shape[1] * X_val.shape[2]))

  model = model_cls(**config)

  model.fit(X_train, y_train)
  val_metrics = evaluate_classic_model(model, X_val, y_val)

  tune.report(
      {
        "accuracy": val_metrics.accuracy,
        "f1": val_metrics.f1,
        "precision": val_metrics.precision,
        "recall": val_metrics.recall
      }
  )

## SVC

In [48]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=SVC,
        X_train=X_train_bal,
        y_train=y_train_bal,
        X_val=X_val_bal,
        y_val=y_val_bal,
    ),
    param_space={
        "C": tune.grid_search([0.1, 0.8, 1, 1.2, 2.0, 10]),
        'class_weight': tune.grid_search(['balanced', None])
    },
    tune_config=tune.TuneConfig(
        num_samples=3,
        metric="accuracy",
        mode="max"
    )
)

results = tuner.fit()
best_metrics = results.get_best_result().metrics
print(f"Best params: {results.get_best_result().config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-07_11-34-04   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 36                                     |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-07_11-34-04
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_11-22-25_127757_2711/artifacts/2025-06-07_11-34-04/train_model_tune_2025-06-07_11-34-04/driver_artifacts`

Trial status: 36 PENDING
Current time: 2025-06-07 11:34:04. Total running time: 0s
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
+-------------------------------------------------

2025-06-07 11:34:27,177	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_11-34-04' in 0.0113s.



Trial train_model_tune_50f96_00032 completed after 1 iterations at 2025-06-07 11:34:27. Total running time: 22s
+-------------------------------------------------------+
| Trial train_model_tune_50f96_00032 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              0.37745 |
| time_total_s                                  0.37745 |
| training_iteration                                  1 |
| accuracy                                      0.93853 |
| f1                                            0.93834 |
| precision                                      0.9394 |
| recall                                        0.93779 |
+-------------------------------------------------------+

Trial train_model_tune_50f96_00033 completed after 1 iterations at 2025-06-07 11:34:27. Total running time: 22s
+-------------------------------------------------------+
| Trial train_model_

In [49]:
run_classic_model(
    SVC,
    results.get_best_result().config,
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_val=X_val_bal,
    y_val=y_val_bal,
    X_test=X_test_bal,
    y_test=y_test_bal,
)

<class 'sklearn.svm._classes.SVC'> & 0.9151 ± 0.0000 & 0.9144 ± 0.0000 0.9187 ± 0.0000 0.9130 ± 0.0000 \
[0.1621236801147461, 0.15827083587646484, 0.15615439414978027, 0.16016149520874023, 0.1550593376159668]
0.1584 ± 0.0026


## RFC

In [50]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=RandomForestClassifier,
        X_train=X_train_bal,
        y_train=y_train_bal,
        X_val=X_val_bal,
        y_val=y_val_bal,
    ),
    param_space={
        "n_estimators": tune.grid_search([10, 50, 75, 100, 150, 200]),
        "max_depth": tune.grid_search([1, 3, 5, 10, None]),
        "criterion": tune.grid_search(['gini', 'log_loss', 'entropy'])
    },
    tune_config=tune.TuneConfig(
        num_samples=3,
        metric="accuracy",
        mode="max"
    )
)

results = tuner.fit()
best_metrics = results.get_best_result().metrics
print(f"Best params: {results.get_best_result().config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              1.10324 |
| time_total_s                                  1.10324 |
| training_iteration                                  1 |
| accuracy                                      0.94978 |
| f1                                             0.9497 |
| precision                                     0.94977 |
| recall                                        0.94964 |
+-------------------------------------------------------+

Trial train_model_tune_e65d0_00054 completed after 1 iterations at 2025-06-07 11:38:47. Total running time: 32s
+-------------------------------------------------------+
| Trial train_model_tune_e65d0_00054 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_thi

2025-06-07 11:41:04,217	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_11-38-15' in 0.0562s.



Trial train_model_tune_e65d0_00268 completed after 1 iterations at 2025-06-07 11:41:04. Total running time: 2min 48s
+-------------------------------------------------------+
| Trial train_model_tune_e65d0_00268 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              1.59208 |
| time_total_s                                  1.59208 |
| training_iteration                                  1 |
| accuracy                                      0.94978 |
| f1                                            0.94969 |
| precision                                     0.94984 |
| recall                                        0.94957 |
+-------------------------------------------------------+

Trial train_model_tune_e65d0_00269 completed after 1 iterations at 2025-06-07 11:41:04. Total running time: 2min 48s
+-------------------------------------------------------+
| Trial tr

In [51]:
run_classic_model(
    RandomForestClassifier,
    results.get_best_result().config,
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_val=X_val_bal,
    y_val=y_val_bal,
    X_test=X_test_bal,
    y_test=y_test_bal,
)

<class 'sklearn.ensemble._forest.RandomForestClassifier'> & 0.9361 ± 0.0017 & 0.9357 ± 0.0018 0.9391 ± 0.0016 0.9344 ± 0.0018 \
[0.07242751121520996, 0.07366657257080078, 0.07471728324890137, 0.07005691528320312, 0.0693807601928711]
0.0720 ± 0.0020


## XGB

In [52]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=XGBClassifier,
        X_train=X_train_bal,
        y_train=y_train_bal,
        X_val=X_val_bal,
        y_val=y_val_bal,
    ),
    param_space={
        "n_estimators": tune.grid_search([10, 50, 75, 100, 150, 200]),
        "max_depth": tune.grid_search([1, 3, 5, 10, None]),
        "booster": tune.grid_search(['gbtree', 'gblinear', 'dart'])
    },
    tune_config=tune.TuneConfig(
        num_samples=3,
        metric="accuracy",
        mode="max"
    )
)

results = tuner.fit()
best_metrics = results.get_best_result().metrics
print(f"Best params: {results.get_best_result().config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-07_11-41-40   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 270                                    |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-07_11-41-40
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_11-22-25_127757_2711/artifacts/2025-06-07_11-41-40/train_model_tune_2025-06-07_11-41-40/driver_artifacts`

Trial status: 200 PENDING
Current time: 2025-06-07 11:41:42. Total running time: 1s
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
+------------------------------------------------

(train_model_tune pid=44112) /usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:41:44] WARNING: /workspace/src/learner.cc:740: 
(train_model_tune pid=44112) Parameters: { "max_depth" } are not used.
(train_model_tune pid=44112) 
(train_model_tune pid=44112)   warnings.warn(smsg, UserWarning)
(train_model_tune pid=44106) 
(train_model_tune pid=44108) 



Trial train_model_tune_60ee9_00004 completed after 1 iterations at 2025-06-07 11:41:44. Total running time: 3s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00004 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                               0.1135 |
| time_total_s                                   0.1135 |
| training_iteration                                  1 |
| accuracy                                      0.94805 |
| f1                                            0.94797 |
| precision                                     0.94803 |
| recall                                        0.94791 |
+-------------------------------------------------------+

Trial train_model_tune_60ee9_00007 completed after 1 iterations at 2025-06-07 11:41:44. Total running time: 4s
+-------------------------------------------------------+
| Trial train_model_tu

(train_model_tune pid=44793) 



Trial train_model_tune_60ee9_00013 completed after 1 iterations at 2025-06-07 11:41:49. Total running time: 8s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00013 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              0.21946 |
| time_total_s                                  0.21946 |
| training_iteration                                  1 |
| accuracy                                      0.94805 |
| f1                                            0.94797 |
| precision                                     0.94803 |
| recall                                        0.94791 |
+-------------------------------------------------------+

Trial train_model_tune_60ee9_00012 completed after 1 iterations at 2025-06-07 11:41:49. Total running time: 8s
+-------------------------------------------------------+
| Trial train_model_tu

...


Trial train_model_tune_60ee9_00262 completed after 1 iterations at 2025-06-07 11:50:34. Total running time: 8min 53s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00262 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              0.72997 |
| time_total_s                                  0.72997 |
| training_iteration                                  1 |
| accuracy                                      0.95152 |
| f1                                            0.95142 |
| precision                                     0.95165 |
| recall                                        0.95124 |
+-------------------------------------------------------+

Trial train_model_tune_60ee9_00263 started with configuration:
+----------------------------------------------------+
| Trial train_model_tune_60ee9_00263 config          |
+-----------

(train_model_tune pid=64664) 
(train_model_tune pid=64664) /usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:51:15] WARNING: /workspace/src/learner.cc:740: 
(train_model_tune pid=64664) Parameters: { "max_depth" } are not used.
(train_model_tune pid=64664)   warnings.warn(smsg, UserWarning)



Trial train_model_tune_60ee9_00265 completed after 1 iterations at 2025-06-07 11:51:15. Total running time: 9min 34s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00265 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              0.79628 |
| time_total_s                                  0.79628 |
| training_iteration                                  1 |
| accuracy                                      0.95152 |
| f1                                            0.95142 |
| precision                                     0.95165 |
| recall                                        0.95124 |
+-------------------------------------------------------+

Trial train_model_tune_60ee9_00245 completed after 1 iterations at 2025-06-07 11:51:19. Total running time: 9min 38s
+-------------------------------------------------------+
| Trial tr

2025-06-07 11:53:01,415	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_11-41-40' in 0.0559s.



Trial train_model_tune_60ee9_00269 completed after 1 iterations at 2025-06-07 11:53:01. Total running time: 11min 20s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00269 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              92.9065 |
| time_total_s                                  92.9065 |
| training_iteration                                  1 |
| accuracy                                      0.95671 |
| f1                                            0.95663 |
| precision                                     0.95686 |
| recall                                        0.95644 |
+-------------------------------------------------------+

Trial status: 270 TERMINATED
Current time: 2025-06-07 11:53:01. Total running time: 11min 20s
Logical resource usage: 1.0/8 CPUs, 0/0 GPUs
Current best trial: 60ee9_00066 with accuracy

In [53]:
run_classic_model(
    XGBClassifier,
    results.get_best_result().config,
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_val=X_val_bal,
    y_val=y_val_bal,
    X_test=X_test_bal,
    y_test=y_test_bal,
)

<class 'xgboost.sklearn.XGBClassifier'> & 0.9455 ± 0.0000 & 0.9452 ± 0.0000 0.9476 ± 0.0000 0.9441 ± 0.0000 \
[0.2169945240020752, 0.18970775604248047, 0.17898130416870117, 0.16037464141845703, 0.153580904006958]
0.1799 ± 0.0226


## Naiva Bayes

In [ ]:
# tuner = tune.Tuner(
#     tune.with_parameters(
#         train_model_tune,
#         model_cls=GaussianNB,
#         X_train=X_train_bal,
#         y_train=y_train_bal,
#         X_val=X_val_bal,
#         y_val=y_val_bal,
#     ),
#     param_space={
#         "n_estimators": tune.grid_search([10, 50, 75, 100, 150, 200]),
#         "max_depth": tune.grid_search([1, 3, 5, 10, None]),
#         "booster": tune.grid_search(['gbtree', 'gblinear', 'dart'])
#     },
#     tune_config=tune.TuneConfig(
#         num_samples=3,
#         metric="accuracy",
#         mode="max"
#     )
# )

# results = tuner.fit()
# best_metrics = results.get_best_result().metrics
# print(f"Best params: {results.get_best_result().config}")
# for k, v in best_metrics.items():
#   if isinstance(v, float):
#     print(f"{k}: {v:.4f}")
#   else:
#     print(f"{k}: {v}")

In [54]:
run_classic_model(
    GaussianNB,
    # results.get_best_result().config,
    {},
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_val=X_val_bal,
    y_val=y_val_bal,
    X_test=X_test_bal,
    y_test=y_test_bal,
)

<class 'sklearn.naive_bayes.GaussianNB'> & 0.8695 ± 0.0000 & 0.8680 ± 0.0000 0.8755 ± 0.0000 0.8664 ± 0.0000 \
[0.004881620407104492, 0.0030670166015625, 0.0031321048736572266, 0.003183603286743164, 0.0030388832092285156]
0.0035 ± 0.0007
